In [1]:
import datetime
from dateutil.relativedelta import relativedelta
import math
import time
import itertools
from contextlib import closing
import numpy as np
import pandas as pd
import MySQLdb
import pymssql
import dfutils

In [2]:
def optimized_execute(query, commit=False):
    '''
    Executes the search query in Northpoint
    '''
    
    with closing(pymssql.connect(host='10.16.1.16', user='readonly_user', password='readonly_user',
                                 database='PnLAppDb')) as np_pnl_conn:
        with closing(np_pnl_conn.cursor()) as np_pnl_cursor:
            np_pnl_cursor.execute(query)
            if commit:
                np_pnl_conn.commit()
                return

            fetched_res = np_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
            return fetched_res

In [3]:
def get_position_calculated_values_history_max_date():  # done
    query = "SELECT MAX(PCVH.TradeDate) FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH "
    results = optimized_execute(query)
    if len(results) == 0:
        return None

    return results[0][0]

In [6]:
def get_position_calculated_values_history(start_date_yyyy_mm_dd=None, end_date_yyyy_mm_dd=None, limit_to_tradegroups=[],
                                           is_tg_names_in_tradar_format=False, fund_code=None, NP_qty_rollup=True):
    if start_date_yyyy_mm_dd is None or end_date_yyyy_mm_dd is None:
        start_date_yyyy_mm_dd = '2017-01-01'
        end_date_yyyy_mm_dd = get_position_calculated_values_history_max_date()
    tg_filter_clause = ""
    if len([tg for tg in limit_to_tradegroups if tg is not None]) > 0:
        tg_filter_clause = " AND PCVH.TradeGroup IN (" + ",".join([("'" + tg + "'") 
                                                                   for tg in limit_to_tradegroups if tg is not None]) + ") "
        if is_tg_names_in_tradar_format: tg_filter_clause = tg_filter_clause.replace('PCVH.TradeGroup',
                                                                                     'SUBSTRING(PCVH.TradeGroup,0,21)')
    fund_code_filtuer_clause = " " if fund_code is None else " AND PCVH.FundCode = '" + fund_code + "' "
    # region query
    # New Fund by added bv Kshitij "WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL":"MACO"
    query = "SELECT " \
            "PCVH.TradeDate, " \
            "ISNULL(PCVH.FundCode, " \
            "CASE F.FundName " \
            "WHEN 'Columbia' THEN 'CAM' " \
            "WHEN 'Litman Gregory' THEN 'LG' " \
            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
            "WHEN 'TransAmerica' THEN 'TAF' " \
            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'WED' " \
            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL' THEN 'MACO' " \
            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
            "END " \
            ") AS FundCode, " \
            "SPT_TRDGRP.SecName AS TradeGroup, " \
            "SPT_TRDGRP.SecName AS TradeGroup_Tradar_Name, " \
            "PCVH.[Marketing GROUP] AS Sleeve,   " \
            "ISNULL(PCVH.[Bucket],'NA') AS Bucket, " \
            "PCVH.TradeGroupId, " \
            "SPT.Ticker, " \
            "SPT.SecType, " \
            "ISNULL(SPT.MarketCapCategory, 'N/A') AS MarketCapCategory, " \
            "PCVH.SecurityID, " \
            "PCVH.DealTermsCash, " \
            "PCVH.DealTermsStock, " \
            "PCVH.DealValue, " \
            "PCVH.DealClosingDate, " \
            "CASE " \
            "WHEN SPT.SecType = 'FxFwd' THEN 0.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Equity Special Situations'  THEN 1.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Opportunistic'  THEN 1.0 " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Merger Arbitrage'  THEN " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') THEN " \
            "CASE PCVH.DealValue WHEN 0 THEN 0.0 ELSE PCVH.DealTermsStock/PCVH.DealValue END " \
            "ELSE 1.0 END " \
            "WHEN SPT.SecType <> 'FxFwd' AND PCVH.[Marketing GROUP] = 'Credit Opportunities'  THEN " \
            "CASE WHEN SPT.SecType IN ('EQ','ExchOpt') THEN 1.0 ELSE 0.0 END " \
            "ELSE NULL END " \
            "AS [Equity Risk Factor], " \
            "PCVH.DV_01, " \
            "PCVH.CR_01, " \
            "PCVH.Adj_CR_01, " \
            "ISNULL(SPT.UltimateCountry,'N/A') AS UltimateCountry, " \
            "ISNULL(PCVH.AlphaHedge,'NA') AS AlphaHedge, " \
            "SUM(PCVH.ExposureLong_USD) AS Exposure_Long, " \
            "SUM(PCVH.ExposureShort_USD) AS Exposure_Short, " \
            "SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD) AS Exposure_Net, " \
            "CASE WHEN NAV.NAV = 0 THEN NULL ELSE 100.0*(SUM(PCVH.ExposureLong_USD+PCVH.ExposureShort_USD)/NAV.NAV) END " \
            "AS [Exposure_Net(%)], " \
            "SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD) AS NetMktVal, " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') " \
            "THEN ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD)) ELSE NULL END AS Capital, " \
            "CASE WHEN ISNULL(PCVH.AlphaHedge,'NA') IN ('Alpha','Alpha Hedge') " \
            "THEN 100.0*(ABS(SUM(PCVH.MktValLong_USD+PCVH.MktValShort_USD))/NAV.NAV) ELSE NULL END AS [Capital % of NAV], " \
            "100.0*SUM(PCVH.Base_Case_NAV_Impact) AS BaseCaseNavImpact, " \
            "100.0*SUM(PCVH.Outlier_NAV_Impact) AS OutlierNavImpact, " \
            "SUM(PCVH.Qty) AS QTY, " \
            "ISNULL(PCVH.TradeGroupLongShortFlag,'NA') AS LongShort, " \
            "ISNULL(SPT.GICSSectorName,'NA') AS Sector, " \
            "ISNULL(SPT.GICSIndustryName,'NA') AS Industry, " \
            "NAV.NAV " \
            "FROM [PnLAppDb].[dbo].[PositionCalculatedValuesHistory] AS PCVH " \
            "INNER JOIN PnLAppDb.dbo.Funds AS F ON PCVH.Portfolio = F.FundID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON PCVH.SecurityID = SPT.ID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TRDGRP ON PCVH.TradeGroupId = SPT_TRDGRP.ID " \
            "LEFT OUTER JOIN PnLAppDb.pnl.DailyNAV AS NAV ON PCVH.Portfolio = NAV.FundId AND PCVH.TradeDate = NAV.[DATE] " \
            "WHERE  TradeDate >= '" + start_date_yyyy_mm_dd + "' AND TradeDate <= '" + end_date_yyyy_mm_dd + "' " \
            + tg_filter_clause + \
            fund_code_filtuer_clause + \
            "GROUP BY " \
            "F.FundName, PCVH.FundCode,PCVH.TradeDate, PCVH.[Marketing GROUP], SPT_TRDGRP.SecName, " \
            "PCVH.TradeGroupId, SPT.Ticker, SPT.BloombergGlobalId, SPT.BloombergGID, SPT.SecType, ISNULL(SPT.MarketCapCategory, 'N/A'), " \
            "PCVH.SecurityId, SPT.UltimateCountry, " \
            "ISNULL(PCVH.AlphaHedge,'NA'),PCVH.TradeGroupLongShortFlag, ISNULL(SPT.GICSSectorName,'NA'), " \
            "ISNULL(SPT.GICSIndustryName,'NA'), ISNULL(PCVH.TradeGroupCatalystRating, 'NA'), " \
            "ISNULL(PCVH.[Bucket],'NA'), PCVH.DealTermsCash, PCVH.DealTermsStock, PCVH.DealValue, PCVH.DealClosingDate, " \
            "PCVH.DV_01, PCVH.CR_01, PCVH.Adj_CR_01, NAV.NAV, PCVH.Analyst " \
            "ORDER BY PCVH.TradeDate "
    # endregion
    results = optimized_execute(query)
    df = pd.DataFrame(results,
                        columns=["Date", "FundCode", "TradeGroup", "TradeGroup_Tradar_Name", "Sleeve", "Bucket",
                                "TradeGroupId", "Ticker", "SecType", "MarketCapCategory", "SecurityId", "DealTermsCash",
                                "DealTermsStock", "DealValue", "DealClosingDate", "Equity_Risk_Factor",
                                "DV01", "CR01", "Adj_CR01", "UltimateCountry",
                                "AlphaHedge", "Exposure_Long", "Exposure_Short", "Exposure_Net",
                                "Exposure_Net(%)", "NetMktVal", "Capital($)", "Capital(%)", "BaseCaseNavImpact", 
                                "OutlierNavImpact", "Qty", "LongShort", "Sector", "Industry", "Fund_NAV"])

    float_cols = ['NetMktVal', 'Capital($)', 'Capital(%)', 'Exposure_Net', 'Exposure_Net(%)', 'BaseCaseNavImpact']
    df[float_cols] = df[float_cols].astype(float)
    df['Equity_Risk_Exp'] = df['Equity_Risk_Factor'].astype(float) * df['Exposure_Net'].astype(float)
    df[['DV01', 'CR01', 'Adj_CR01']] = df[['DV01', 'CR01', 'Adj_CR01']].fillna(0).astype(float)
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x)) 
    df["DealClosingDate"] = df["DealClosingDate"].apply(lambda x: pd.to_datetime(x)) 

    as_of_dt = datetime.datetime.strptime(end_date_yyyy_mm_dd, '%Y-%m-%d')

    def duration_calc(days):
        if days <= 90: return '0M-3M'
        if days <= 180: return '3M-6M'
        if days <= 365: return '6M-12M'
        return 'Yr+'

    df["DealDuration"] = df["DealClosingDate"].apply(lambda x: None if pd.isnull(x) else 
                                                     duration_calc((as_of_dt - pd.to_datetime(x)).days))

    # region ADJUSTING FOR POSTED TRADES AFTER TRADE DATE - AFFECTING QTY ONLY
    # adjust for end_date_yyyy_mm_dd - take trades with tradedate = end_date_yyyy_mm_dd and posted_date 
    #AFTER end_date_yyyy_mm_dd
    # adjust the Qty into the pcvh_df
    if NP_qty_rollup:
        qty_adjust_query = "SELECT " \
                            "A.TradeDate AS [DATE], " \
                            "CASE A.Fund " \
                            "WHEN 'Columbia' THEN 'CAM' " \
                            "WHEN 'Litman Gregory' THEN 'LG' " \
                            "WHEN 'The Arbitrage Credit Opportunities Fund' THEN 'TACO' " \
                            "WHEN 'The Arbitrage Event-Driven Fund' THEN 'AED' " \
                            "WHEN 'The Arbitrage Fund' THEN 'ARB' " \
                            "WHEN 'TransAmerica' THEN 'TAF' " \
                            "WHEN 'WIC Arbitrage Partners' THEN 'WIC' " \
                            "WHEN 'The Arbitrage Tactical Equity Fund' THEN 'TAQ' " \
                            "WHEN 'Water Island Event-Driven Fund' THEN 'WED' " \
                            "WHEN 'Water Island Capital Lev Arb Fund' THEN 'LEV' " \
                            "WHEN 'WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL COMMINGLED MASTER FUND LP' THEN 'MACO' " \
                            "WHEN 'Morningstar Alternatives Fund' THEN 'MALT' " \
                            "END AS FundCode, " \
                            "A.TradeGroupId, " \
                            "A.SecurityId, " \
                            "SUM((CASE SPT.SecType WHEN 'ExchOpt' THEN 100.0 ELSE 1.0 END)*A.Shares) AS Qty " \
                            "FROM " \
                            "[PnLAppDb].[dbo].[vTradesFlatView] AS A " \
                            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT ON A.SecurityId = SPT.ID " \
                            "WHERE PostedDate >= DATEADD(DAY,1,TradeDate) AND TradeDate >= '" \
                            + start_date_yyyy_mm_dd + "' " \
                            "GROUP BY A.TradeDate, A.SecurityId, A.Ticker, A.Fund, A.TradeGroupId, SPT.SecType "
        
        results = optimized_execute(qty_adjust_query)
        qty_adjust_df = pd.DataFrame(results, columns=['Date', 'FundCode', 'TradeGroupId', 'SecurityId', 'Qty_adj'])
        qty_adjust_df[['TradeGroupId', 'SecurityId', 'Qty_adj']] = qty_adjust_df[['TradeGroupId',
                                                                                  'SecurityId', 'Qty_adj']].astype(float)
        qty_adjust_df['Date'] = qty_adjust_df['Date'].apply(lambda x: pd.to_datetime(x))

        df[['TradeGroupId', 'SecurityId', 'Qty']] = df[['TradeGroupId', 'SecurityId', 'Qty']].astype(float)
        df = pd.merge(df, qty_adjust_df, how='left', on=['Date', 'FundCode', 'TradeGroupId', 'SecurityId'])
        df['Qty'] = df['Qty'] + df['Qty_adj'].fillna(0)
        del df['Qty_adj']
    # endregion
    print 'PCVH retrieval completed....'
    return df

In [7]:
tg = "GG - NEM"
tradar_tg_name = tg[:20]
fund_code = "ARB" #CAN CHANGE NAME TO CODE AND VICE VERSA USING QUERY
tg_pcvh = get_position_calculated_values_history(limit_to_tradegroups=[tradar_tg_name],
                                                 is_tg_names_in_tradar_format=True, fund_code=fund_code)
tg_pcvh

PCVH retrieval completed....


,Date,FundCode,TradeGroup,TradeGroup_Tradar_Name,Sleeve,Bucket,TradeGroupId,Ticker,SecType,MarketCapCategory,...,Capital(%),BaseCaseNavImpact,OutlierNavImpact,Qty,LongShort,Sector,Industry,Fund_NAV,Equity_Risk_Exp,DealDuration
0,2019-01-14,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Optimized,94230.0,GG US,EQ,Mid Cap,...,0.2137,0.0,0.00000,358795.0,Long,Materials,Metals & Mining,1742520435.020000,3.730233e+06,0M-3M
1,2019-01-14,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Optimized,94230.0,NEM US,EQ,Large Cap,...,NaN,0.0,0.00000,-117696.0,Long,Materials,Metals & Mining,1742520435.020000,-3.740379e+06,0M-3M
2,2019-01-15,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Conviction,94230.0,GG US,EQ,Mid Cap,...,0.3964,0.0,0.00000,676556.0,Long,Materials,Metals & Mining,1740819438.670000,6.911876e+06,0M-3M
3,2019-01-15,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Conviction,94230.0,NEM US,EQ,Large Cap,...,NaN,0.0,0.00000,-221923.0,Long,Materials,Metals & Mining,1740819438.670000,-6.884051e+06,0M-3M
4,2019-01-16,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Conviction,94230.0,GG US,EQ,Mid Cap,...,0.6419,0.0,0.00000,1095581.0,Long,Materials,Metals & Mining,1754324716.740000,1.128601e+07,0M-3M
5,2019-01-16,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Conviction,94230.0,NEM US,EQ,Large Cap,...,NaN,0.0,0.00000,-359361.0,Long,Materials,Metals & Mining,1754324716.740000,-1.134143e+07,0M-3M
6,2019-01-17,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Conviction,94230.0,GG US,EQ,Mid Cap,...,0.6452,0.0,0.00000,1095581.0,Long,Materials,Metals & Mining,1755766758.000000,1.122216e+07,0M-3M
7,2019-01-17,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Conviction,94230.0,NEM US,EQ,Large Cap,...,NaN,0.0,0.00000,-359361.0,Long,Materials,Metals & Mining,1755766758.000000,-1.136299e+07,0M-3M
8,2019-01-18,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Conviction,94230.0,GG US,EQ,Mid Cap,...,0.6483,0.0,0.00000,1117898.0,Long,Materials,Metals & Mining,1788073643.010000,1.148459e+07,0M-3M
9,2019-01-18,ARB,GG - NEM,GG - NEM,Merger Arbitrage,Conviction,94230.0,NEM US,EQ,Large Cap,...,NaN,0.0,0.00000,-366682.0,Long,Materials,Metals & Mining,1788073643.010000,-1.164949e+07,0M-3M


In [8]:
#Current Holdings of TradeGroup Data Table
pcv_df = tg_pcvh[tg_pcvh['Date'] == tg_pcvh['Date'].max()].copy()
pcv_df['Date'] = pcv_df['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
pcv_df['DeltaAdj Net(% of NAV)'] = 100.0*(pcv_df['Exposure_Net'].astype(float)/pcv_df['Fund_NAV'].astype(float))
pcv_df['DeltaAdj GrossExp'] = np.abs(pcv_df['Exposure_Net'])
pcv_df['DeltaAdj GrossExp(% of NAV)'] = 100.0*(pcv_df['DeltaAdj GrossExp'].astype(float)/pcv_df['Fund_NAV'].astype(float))
pcv_df = pcv_df.rename(columns = {'Exposure_Short':'DeltaAdj Exp Short', 'Exposure_Long':'DeltaAdj Exp Long', 
                                  'Exposure_Net':'DeltaAdj Net'})
pcv_df = pcv_df[['Date', 'FundCode', 'Sleeve', 'TradeGroup', 'Ticker', 'Qty', 'AlphaHedge', 'DeltaAdj Exp Long', 
                 'DeltaAdj Exp Short', 'DeltaAdj Net','DeltaAdj Net(% of NAV)', 'DeltaAdj GrossExp',
                 'DeltaAdj GrossExp(% of NAV)','NetMktVal', 'Capital($)', 'Capital(%)', 'BaseCaseNavImpact', 
                 'OutlierNavImpact']]
cols = ['Qty', 'DeltaAdj Exp Long', 'DeltaAdj Exp Short', 'DeltaAdj Net','DeltaAdj Net(% of NAV)',
        'DeltaAdj GrossExp','DeltaAdj GrossExp(% of NAV)','NetMktVal', 'Capital($)', 'Capital(%)',
        'BaseCaseNavImpact', 'OutlierNavImpact']
cols2 = ['Date', 'FundCode', 'Sleeve', 'TradeGroup', 'Ticker', 'AlphaHedge']
pcv_df[cols] = pcv_df[cols].astype(float)
pcv_df[cols2] = pcv_df[cols2].astype(str) #convert from unicode string
pcv_df = pcv_df.set_index('Ticker')
current_holdings_dict = pcv_df.to_dict('index')
current_holdings_dict

{'GG    190418P00010000': {'AlphaHedge': 'Alpha Hedge',
  'BaseCaseNavImpact': 0.0,
  'Capital($)': 2068.0,
  'Capital(%)': 0.0001,
  'Date': '2019-04-03',
  'DeltaAdj Exp Long': 0.0,
  'DeltaAdj Exp Short': -38896.1832,
  'DeltaAdj GrossExp': 38896.1832,
  'DeltaAdj GrossExp(% of NAV)': 0.002218078317354462,
  'DeltaAdj Net': -38896.1832,
  'DeltaAdj Net(% of NAV)': -0.002218078317354462,
  'FundCode': 'ARB',
  'NetMktVal': 2068.0,
  'OutlierNavImpact': 0.0,
  'Qty': 37600.0,
  'Sleeve': 'Merger Arbitrage',
  'TradeGroup': 'GG - NEM'},
 'GG US': {'AlphaHedge': 'Alpha',
  'BaseCaseNavImpact': 0.0,
  'Capital($)': 35009395.2,
  'Capital(%)': 1.9964,
  'Date': '2019-04-03',
  'DeltaAdj Exp Long': 35009395.2,
  'DeltaAdj Exp Short': 0.0,
  'DeltaAdj GrossExp': 35009395.2,
  'DeltaAdj GrossExp(% of NAV)': 1.9964318863248613,
  'DeltaAdj Net': 35009395.2,
  'DeltaAdj Net(% of NAV)': 1.9964318863248613,
  'FundCode': 'ARB',
  'NetMktVal': 35009395.2,
  'OutlierNavImpact': 0.0,
  'Qty': 30390

In [9]:
def get_NAV_df(fund_name=None):  # didn't find required fields (NAV)
    if fund_name = None:
        fund_name = ''
        
    query = "SELECT [DATE],NAV " \
            "FROM PnLAppDb.pnl.DailyNAV AS A " \
            "INNER JOIN PnLAppDb.dbo.Funds AS B ON A.FundId = B.FundID " \
            "WHERE B.FundName = '" + fund_name + "' " \
            "ORDER BY [DATE]"

    results = optimized_execute(query)
    df = pd.DataFrame(results, columns=["Date", "NAV"])
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df.index.name = 'Date'
    df = df.sort_index()
    df['NAV'] = df['NAV'].astype(float)
    idx = pd.date_range(df['Date'].min(), df['Date'].max())
    df = df.reindex(idx, fill_value=np.nan)
    del df['Date']
    df.reset_index(level=0, inplace=True)
    df.rename(columns={"index": "Date"}, inplace=True)
    df.index.name = 'Date'
    df.ffill(inplace = True)
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    # df.reset_index(inplace=True)
    df = df.reset_index(level=0, drop=True).reset_index()  # Added by Kshitij
    del df['index']
    return df[df['Date'] <= (datetime.datetime.today() - datetime.timedelta(days=1))]

In [10]:
fund_name = "Water Island Event-Driven Fund"
fund_nav_df = get_NAV_df(fund_name)
if len(fund_nav_df) == 0: 
     print("Empty NAV timeseries for fund " + fund_name)

In [11]:
#Capital as (%) of NAV Time Series
capital_df = tg_pcvh[tg_pcvh['AlphaHedge'].isin(['Alpha',
                                                 'Alpha Hedge'])][['Date',
                                                                   'NetMktVal']].groupby('Date').agg(lambda x: 
                                                                                                     sum(abs(x))).reset_index()
if len(capital_df) > 0:
    capital_df = pd.merge(capital_df, fund_nav_df, how='inner', on=['Date'])
    capital_df['CapitalAsPctOfNAV'] = 100.0*(capital_df['NetMktVal'].astype(float)/capital_df['NAV'].astype(float))

capital_as_pct_nav_dict = pd.Series(capital_df.CapitalAsPctOfNAV.values,
                                    index=capital_df.Date.dt.strftime('%Y-%m-%d')).to_dict()
capital_as_pct_nav_dict

{'2019-01-14': 46.736812819439386,
 '2019-01-15': 93.77945182108289,
 '2019-01-16': 153.0528336806959,
 '2019-01-17': 153.94613815743148,
 '2019-01-18': 157.53777365423753,
 '2019-01-22': 159.20885900640397,
 '2019-01-23': 159.5126927067979,
 '2019-01-24': 159.20885900640397,
 '2019-01-25': 176.85798061349175,
 '2019-01-28': 249.1054893863862,
 '2019-01-29': 254.46510234792675,
 '2019-01-30': 277.2414636496498,
 '2019-01-31': 307.3404499735144,
 '2019-02-01': 303.2206047996067,
 '2019-02-04': 305.14319921409697,
 '2019-02-05': 306.51648093873285,
 '2019-02-06': 304.04457383438825,
 '2019-02-07': 302.39663576482513,
 '2019-02-08': 303.4952611445339,
 '2019-02-11': 301.2980103851164,
 '2019-02-12': 374.07922089931145,
 '2019-02-13': 342.1240214150426,
 '2019-02-14': 343.3946770191839,
 '2019-02-15': 343.71234092021916,
 '2019-02-19': 395.04634206894855,
 '2019-02-20': 431.6696132882648,
 '2019-02-21': 431.48217533199545,
 '2019-02-22': 229.59537342540045,
 '2019-02-25': 228.1513773661211

In [12]:
#NetExp (% of NAV)
tot_net_exp_df = tg_pcvh[['Date', 'Exposure_Net']].groupby('Date').sum().reset_index()
tot_net_exp_df = tot_net_exp_df.rename(columns = {'Exposure_Net':'Total Exposure'}).sort_values(by = 'Date')
tot_net_exp_df = pd.merge(tot_net_exp_df,fund_nav_df,how = 'inner',on = ['Date'])
tot_net_exp_df['NetExpAsPctOfNAV'] = 100.0*(tot_net_exp_df['Total Exposure'].astype(float)/tot_net_exp_df['NAV'].astype(float))

total_net_exp_dict = pd.Series(tot_net_exp_df.NetExpAsPctOfNAV.values, index = tot_net_exp_df.Date.dt.strftime('%Y-%m-%d')).to_dict()


In [13]:
#GrossExp (% of NAV)
tot_gross_exp_df = tg_pcvh[['Date', 'Exposure_Net']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index()
tot_gross_exp_df = tot_gross_exp_df.rename(columns={'Exposure_Net':'Total Gross Exposure'}).sort_values(by='Date')
tot_gross_exp_df = pd.merge(tot_gross_exp_df, fund_nav_df, how='inner',on=['Date'])
tot_gross_exp_df['GrossExpAsPctOfNAV'] = 100.0*(tot_gross_exp_df['Total Gross Exposure'].astype(float)
                                                /tot_gross_exp_df['NAV'].astype(float))

total_gross_exp_dict = pd.Series(tot_gross_exp_df.GrossExpAsPctOfNAV.values,
                                 index=tot_gross_exp_df.Date.dt.strftime('%Y-%m-%d')).to_dict()

In [14]:
if len(tg_pcvh) > 0:
    alpha_net_exp = tg_pcvh[tg_pcvh['AlphaHedge'] == 'Alpha'][['Date',
                                                               'Exposure_Net']].groupby('Date').sum().reset_index()
    alphahedge_net_exp = tg_pcvh[tg_pcvh['AlphaHedge'] == 'Alpha Hedge'][['Date',
                                                                          'Exposure_Net']].groupby('Date').sum().reset_index()
    hedge_net_exp = tg_pcvh[tg_pcvh['AlphaHedge'] == 'Hedge'][['Date',
                                                               'Exposure_Net']].groupby('Date').sum().reset_index()

    #### EXPOSURE BREAK DOWN TO ALPHA/HEDGE
    if len(alpha_net_exp) > 0:
        alpha_net_exp = pd.merge(alpha_net_exp, fund_nav_df, how='inner', on=['Date'])
        alpha_net_exp['Exposure_Net'] = 100.0*(alpha_net_exp['Exposure_Net'].astype(float)
                                               /alpha_net_exp['NAV'].astype(float))
        alpha_net_exp_dict = pd.Series(alpha_net_exp.Exposure_Net.values,
                                       index=alpha_net_exp.Date.dt.strftime('%Y-%m-%d')).to_dict()
        
    if len(alphahedge_net_exp) > 0:
        alphahedge_net_exp = pd.merge(alphahedge_net_exp, fund_nav_df, how='inner', on=['Date'])
        alphahedge_net_exp['Exposure_Net'] = 100.0*(alphahedge_net_exp['Exposure_Net'].astype(float)
                                                    /alphahedge_net_exp['NAV'].astype(float))
        alphahedge_net_exp_dict = pd.Series(alphahedge_net_exp.Exposure_Net.values,
                                            index=alphahedge_net_exp.Date.dt.strftime('%Y-%m-%d')).to_dict()
        
    if len(hedge_net_exp) > 0:
        hedge_net_exp = pd.merge(hedge_net_exp, fund_nav_df, how='inner', on=['Date'])
        hedge_net_exp['Exposure_Net'] = 100.0*(hedge_net_exp['Exposure_Net'].astype(float)
                                               /hedge_net_exp['NAV'].astype(float))
        hedge_net_exp_dict = pd.Series(hedge_net_exp.Exposure_Net.values,
                                       index=hedge_net_exp.Date.dt.strftime('%Y-%m-%d')).to_dict()

In [15]:
def get_spread_history(tradar_tg_name):
    query = "SELECT DISTINCT " \
            "PCVH.TradeDate, " \
            "SPT_TRDGRP.SecName AS TradeGroup, " \
            "SPT_TRDGRP.SecName AS TradeGroup_TradarName, " \
            "PCVH.TradeGroupId, " \
            "PCVH.AllInSpread, " \
            "PCVH.DealValue, " \
            "100.0*(PCVH.AllInSpread/PCVH.DealValue) AS spread " \
            "FROM PnLAppDb.dbo.PositionCalculatedValuesHistory AS PCVH " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TRDGRP ON PCVH.TradeGroupId = SPT_TRDGRP.ID " \
            "INNER JOIN SecurityMaster.dbo.SecurityPivotTable AS SPT_TARGET ON PCVH.SecurityID= SPT_TARGET.ID " \
            "WHERE  SPT_TRDGRP.TargetTickers LIKE ('%'+SPT_TARGET.Ticker+'%') AND PCVH.TradeDate >= '2015-11-09' " \
            "AND PCVH.ConsolidatedMarketingGroup = 'Merger Arbitrage' AND PCVH.DealValue <> 0 " \
            "AND PCVH.AllInSpread IS NOT NULL " \
            "AND TradeGroup = '" + tradar_tg_name + "' " \
            "ORDER BY TradeDate "
    
    results = optimized_execute(query)
    columns = ["Date", "TradeGroup", "TradeGroup_TradarName", "TradeGroupId", "AllInSpread", "DealValue",
                "Spread(%)"]
    df = pd.DataFrame(results, columns=columns)
    df["Date"] = df["Date"].apply(lambda x: pd.to_datetime(x))  # convert timestamp to period. convention
    return df

spreads_history_df = get_spread_history(tradar_tg_name)

In [16]:
#Spread (%)
spreads_ts_df = None
if tradar_tg_name in spreads_history_df['TradeGroup_TradarName'].unique():
    spreads_ts_df = spreads_history_df[spreads_history_df['TradeGroup_TradarName'] 
                                       == tradar_tg_name][['Date','Spread(%)']].sort_values(by='Date')
    spreads_ts_df = spreads_ts_df.rename(columns={'Spread(%)':'SpreadAsPct'}).sort_values(by='Date')
    spreads_ts_df['SpreadAsPct'] = spreads_ts_df['SpreadAsPct'].astype(float)
    spread_pct_dict = pd.Series(spreads_ts_df.SpreadAsPct.values,
                                index=spreads_ts_df.Date.dt.strftime('%Y-%m-%d')).to_dict() 

In [17]:
DB_NAME = 'wic'
WIC_DB_HOST = 'wic-risk-database.cwi02trt7ww1.us-east-1.rds.amazonaws.com'
DB_USER = 'aduprey'
DB_PASSWORD = 'aduprey'

def wic_optimized_execute(query, commit=False, retrieve_column_names=False, connection_timeout=250, extra_values=None):
    with closing(MySQLdb.connect(host=WIC_DB_HOST, user=DB_USER, passwd=DB_PASSWORD, db=DB_NAME)) as wic_cnx:
        with closing(wic_cnx.cursor()) as wic_cursor:

            if extra_values is not None:
                wic_cursor.execute(query, (extra_values,))
            else:
                wic_cursor.execute(query)
            if commit:
                wic_cnx.commit()
                return
            fetched_res = wic_cursor.fetchall()  # fetch (and discard) remaining rows

            if retrieve_column_names:
                return fetched_res, [i[0] for i in wic_cursor.description]

            return fetched_res

def get_tradegroups_pnl_cache(fundcode=None, tg=None):
    query = "SELECT `DATE`, Fund, TradeGroup, pnl, cumpnl FROM " + DB_NAME + ".`tradegroups_pnl_cache`"
    if fundcode is not None and tg is not None:
        query += " WHERE Fund = '" + fundcode + "' AND TradeGroup ='" + tg + "' "
    elif fundcode is not None:
        query += " WHERE Fund = '" + fundcode + "' "
    elif tg is not None:
        query += " WHERE TradeGroup = '" + tg + "' "
    else:
        query += ""

    res = wic_optimized_execute(query)
    cols = ['Date', 'Fund', 'TradeGroup', 'Total P&L', 'Cumulative Total P&L']
    df = pd.DataFrame(list(res), columns=cols)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    df['Total P&L'] = df['Total P&L'].astype(float)
    df['Cumulative Total P&L'] = df['Cumulative Total P&L'].astype(float)
    return df

In [18]:
tg_pnl_df = get_tradegroups_pnl_cache(fundcode=fund_code, tg=tradar_tg_name)
#tg_pnl_df

In [19]:
def tradar_optimized_execute(query, commit=False):
    try:
        with closing(pymssql.connect(host='NYDC-WTRTRD01', user='paz', password='Welcome2',
                                     database='TradarBE')) as trdr_pnl_conn:
            with closing(trdr_pnl_conn.cursor()) as trdr_pnl_cursor:
                trdr_pnl_cursor.execute(query)
                if commit:
                    trdr_pnl_conn.commit()
                    return

                fetched_res = trdr_pnl_cursor.fetchall()  # fetch (and discard) remaining rows\
                return fetched_res
    except Exception as e:
        print e

In [20]:
def get_securities_pnl_by_tradegroup_and_fund(tg, fund_code, start_date_yyyymmdd=None, end_date_yyyymmdd=None,
                                              rollup_pnl=False):
    if start_date_yyyymmdd is None: start_date_yyyymmdd = '20170101'
    if end_date_yyyymmdd is None:
        now = datetime.datetime.now()
        slicer = dfutils.df_slicer()
        end_date = slicer.prev_n_business_days(1, now)
        end_date_yyyymmdd = end_date.strftime('%Y%m%d')

    # region rollup query
    rollup_sql_query = "DECLARE @Holidays TABLE ([Date] DATE); " \
                        "DECLARE @StartDt INT; " \
                        "DECLARE @EndDt INT; " \
                        "INSERT INTO @Holidays VALUES " \
                        "('2015-01-01'), " \
                        "('2015-01-19'), " \
                        "('2015-02-16'), " \
                        "('2015-04-03'), " \
                        "('2015-05-25'), " \
                        "('2015-07-03'), " \
                        "('2015-09-07'), " \
                        "('2015-10-12'), " \
                        "('2015-11-11'), " \
                        "('2015-11-26'), " \
                        "('2015-12-25'), " \
                        "('2016-01-01'), " \
                        "('2016-01-18'), " \
                        "('2016-02-15'), " \
                        "('2016-03-25'), " \
                        "('2016-05-30'), " \
                        "('2016-07-04'), " \
                        "('2016-09-05'), " \
                        "('2016-11-24'), " \
                        "('2016-12-25'), " \
                        "('2017-01-02'), " \
                        "('2017-01-16'), " \
                        "('2017-02-20'), " \
                        "('2017-04-14'), " \
                        "('2017-05-29'), " \
                        "('2017-07-04'), " \
                        "('2017-09-04'), " \
                        "('2017-10-09'), " \
                        "('2017-11-23'), " \
                        "('2017-12-25'); " \
                        "SET @StartDt = " + start_date_yyyymmdd + "; " \
                        "SET @EndDt = " + end_date_yyyymmdd + "; " \
                        "WITH X AS( " \
                        "SELECT " \
                        "CAST(CONVERT(varchar(8), A.timeKey) AS DATE) as [Date], " \
                        "F.fund, " \
                        "B.strat AS TradeGroup, " \
                        "S.ticker, " \
                        "ST.groupingName as sectype, " \
                        "SUM(pnlFC) as PNL " \
                        "FROM performanceAttributionFact AS A " \
                        "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
                        "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
                        "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
                        "INNER JOIN SecType AS ST ON ST.sectype = S.sectype " \
                        "WHERE A.timeKey >= @StartDt and A.timeKey <= @EndDt and B.strat = '" \
                        + tg + "' and F.fund = '" + fund_code + "' " \
                        "GROUP BY A.timeKey, B.strat, F.fund, S.ticker, ST.groupingName " \
                        ") " \
                        ", PNL AS ( " \
                        "SELECT " \
                        "X.[Date],  " \
                        "CASE " \
                        "WHEN DATEPART(dw,X.[Date]) IN (7,1) THEN 'WEEKEND' " \
                        "WHEN X.[Date] IN (SELECT * FROM @Holidays) THEN 'HOLIDAY' " \
                        "ELSE 'BUSINESS DAY' END AS [WeekDay], " \
                        "X.Fund, " \
                        "X.TradeGroup, " \
                        "X.Ticker, " \
                        "X.sectype, " \
                        "X.PNL " \
                        "FROM X " \
                        "), " \
                        "Position2EndDate AS ( " \
                        "SELECT " \
                        "X.fund, " \
                        "X.TradeGroup, " \
                        "X.ticker, " \
                        "X.sectype, " \
                        "MAX(X.[Date]) AS MaxDate, " \
                        "CASE DATEPART(dw,MAX(X.[Date])) " \
                        "WHEN 6 THEN DATEADD(DAY,3,MAX(X.[Date])) " \
                        "WHEN 7 THEN DATEADD(DAY,2,MAX(X.[Date])) " \
                        "ELSE DATEADD(DAY,1,MAX(X.[Date])) END AS NextBusinessDay " \
                        "FROM PNL AS X " \
                        "GROUP BY X.fund, X.TradeGroup, X.ticker, X.sectype " \
                        "), " \
                        "Position2EndDate_BD AS ( " \
                        "SELECT " \
                        "X.fund, " \
                        "X.TradeGroup, " \
                        "X.ticker, " \
                        "X.sectype, " \
                        "X.MaxDate, " \
                        "CASE " \
                        "WHEN NextBusinessDay IN (SELECT * FROM @Holidays) THEN " \
                        "CASE DATEPART(dw,NextBusinessDay) " \
                        "WHEN 6 THEN DATEADD(DAY,3,NextBusinessDay) " \
                        "ELSE DATEADD(DAY,1,NextBusinessDay) END " \
                        "ELSE NextBusinessDay " \
                        "END AS NextBusinessDay_Holiday_Proof " \
                        "FROM Position2EndDate AS X " \
                        "), " \
                        "PNL_DT AS ( " \
                        "SELECT " \
                        "A1.[Date], " \
                        "A1.[WeekDay],  " \
                        "A1.fund, " \
                        "A1.TradeGroup, " \
                        "A1.ticker, " \
                        "A1.sectype, " \
                        "A1.PNL, " \
                        "MAX(A2.[Date]) AS [Last Business Day], " \
                        "DATEDIFF(DAY,MAX(A2.[Date]),A1.[Date]) AS [Last BD days diff] " \
                        "FROM PNL AS A1 " \
                        "LEFT OUTER JOIN PNL AS A2 ON A1.fund = A2.fund and A1.TradeGroup = A2.TradeGroup " \
                        "AND A1.ticker = A2.ticker AND A1.sectype = A2.sectype AND A2.[Date] < A1.[Date] " \
                        "AND A2.[WeekDay] = 'BUSINESS DAY' " \
                        "GROUP BY A1.[Date],A1.[WeekDay],A1.fund, A1.TradeGroup, A1.ticker, A1.sectype, A1.PNL " \
                        "), " \
                        "T AS ( " \
                        "SELECT " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN B.NextBusinessDay_Holiday_Proof ELSE A1.[Date] END AS [Date], " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN 'BUSINESS DAY' ELSE A1.[WeekDay] END AS [WeekDay], " \
                        "A1.fund, " \
                        "A1.TradeGroup, " \
                        "A1.ticker, " \
                        "A1.sectype, " \
                        "A1.[Last Business Day], " \
                        "A1.PNL, " \
                        "ISNULL(SUM(A2.PNL),0) AS ROLLED_UP_PNL, " \
                        "A1.PNL + ISNULL(SUM(A2.PNL),0) AS TOT_PNL,  " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN 'Closed on non-BD. Shifted to next BD' ELSE NULL END AS 'Notes' " \
                        "FROM PNL_DT AS A1 " \
                        "LEFT OUTER JOIN PNL_DT AS A2 " \
                        "ON A1.fund = A2.fund AND A1.TradeGroup = A2.TradeGroup AND A1.ticker = A2.ticker " \
                        "AND A1.sectype = A2.sectype AND " \
                        "(A2.[Date] < A1.[Date] AND (A2.[Date] > A1.[Last Business Day] OR A1.[Last Business Day] IS NULL)) " \
                        "AND (A1.[Last BD days diff] <= 4 OR A1.[Last BD days diff] IS NULL) " \
                        "INNER JOIN Position2EndDate_BD AS B ON A1.fund = B.fund AND A1.TradeGroup = B.TradeGroup " \
                        "AND A1.ticker = B.ticker AND A1.sectype = B.sectype " \
                        "WHERE A1.[WeekDay] = 'BUSINESS DAY' OR A1.[Date] = B.MaxDate " \
                        "GROUP BY A1.[Date],A1.[WeekDay], A1.fund,A1.TradeGroup, A1.ticker, " \
                        "A1.sectype, A1.[Last Business Day],A1.[Last BD days diff],A1.PNL, B.MaxDate, " \
                        "B.NextBusinessDay_Holiday_Proof " \
                        ") " \
                        "SELECT " \
                        "T.[Date],  " \
                        "T.fund, " \
                        "T.TradeGroup, " \
                        "T.ticker, " \
                        "T.sectype, " \
                        "T.TOT_PNL " \
                        "FROM T " \
                        "ORDER BY T.fund, T.TradeGroup, T.ticker,T.sectype, T.[Date] "
        
    # endregion
    # region non-rollup query
    # don't take today's pnl - tradar's garbage data
    non_rollup_sql_query = \
        "SELECT " \
        "CAST(CONVERT(VARCHAR(8), A.timeKey) AS DATE) AS [DATE], " \
        "F.fund, " \
        "B.strat AS TradeGroup, " \
        "S.ticker, " \
        "ST.groupingName AS sectype, " \
        "SUM(pnlFC) AS PNL " \
        "FROM performanceAttributionFact AS A " \
        "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
        "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
        "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
        "INNER JOIN SecType AS ST ON ST.sectype = S.sectype " \
        "WHERE A.timeKey >= " + start_date_yyyymmdd + " AND A.timeKey <= " \
        + end_date_yyyymmdd + " AND B.strat = '" + tg + "' AND F.fund = '" + fund_code + "' " \
        "GROUP BY A.timeKey, B.strat, F.fund,S.ticker,ST.groupingName " \
        "ORDER BY F.fund, B.strat,S.ticker,A.timeKey "
    # endregion
    query = rollup_sql_query if rollup_pnl else non_rollup_sql_query
    cols = ['Date', 'Fund', 'TradeGroup', 'Ticker', 'SecType', 'Total P&L']
    try:
        query_result = tradar_optimized_execute(query)
    except:
        return pd.DataFrame(columns=cols)

    df = pd.DataFrame(query_result, columns=cols)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    df['Total P&L'] = df['Total P&L'].astype(float)
    df['Cumulative Total P&L'] = None
    return df

In [21]:
securities_pnl_df = get_securities_pnl_by_tradegroup_and_fund(tradar_tg_name, fund_code, rollup_pnl=True)
securities_pnl_df

,Date,Fund,TradeGroup,Ticker,SecType,Total P&L,Cumulative Total P&L
0,2019-03-01,ARB,GG - NEM,GG 190418P00010000,Option,-1653.6156,None
1,2019-03-04,ARB,GG - NEM,GG 190418P00010000,Option,-11934.4849,None
2,2019-03-05,ARB,GG - NEM,GG 190418P00010000,Option,-4018.5880,None
3,2019-03-06,ARB,GG - NEM,GG 190418P00010000,Option,14160.5992,None
4,2019-03-07,ARB,GG - NEM,GG 190418P00010000,Option,6902.0000,None
5,2019-03-08,ARB,GG - NEM,GG 190418P00010000,Option,-8627.5000,None
6,2019-03-11,ARB,GG - NEM,GG 190418P00010000,Option,-20817.1403,None
7,2019-03-12,ARB,GG - NEM,GG 190418P00010000,Option,-3315.0000,None
8,2019-03-13,ARB,GG - NEM,GG 190418P00010000,Option,0.0000,None
9,2019-03-14,ARB,GG - NEM,GG 190418P00010000,Option,1989.0000,None


In [23]:
securities_pnl_df[securities_pnl_df.Ticker == "NEM US"]

,Date,Fund,TradeGroup,Ticker,SecType,Total P&L,Cumulative Total P&L
82,2019-01-14,ARB,GG - NEM,NEM US,Equity,7.564020e+04,None
83,2019-01-15,ARB,GG - NEM,NEM US,Equity,7.230349e+04,None
84,2019-01-16,ARB,GG - NEM,NEM US,Equity,-1.100685e+05,None
85,2019-01-17,ARB,GG - NEM,NEM US,Equity,-2.156166e+04,None
86,2019-01-18,ARB,GG - NEM,NEM US,Equity,-5.409679e+04,None
87,2019-01-21,ARB,GG - NEM,NEM US,Equity,0.000000e+00,None
88,2019-01-22,ARB,GG - NEM,NEM US,Equity,-1.210051e+05,None
89,2019-01-23,ARB,GG - NEM,NEM US,Equity,-3.666820e+04,None
90,2019-01-24,ARB,GG - NEM,NEM US,Equity,7.333640e+03,None
91,2019-01-25,ARB,GG - NEM,NEM US,Equity,-3.636475e+05,None


In [25]:
#Pull Prices of TradeGroup Equities
#tg_securities_price_dfs = []
#tg_securities = securities_pnl_df[securities_pnl_df['SecType']!='Option']['Ticker'].unique()

#tg_pnl_df = get_tradegroups_pnl_cache(fundcode = fund_code, tg =  tradar_tg_name)

### TRIM THE ZEROS FROM P&L
#zero_pnl_num_of_days_trimmed = len(tg_pnl_df) - len(np.trim_zeros(tg_pnl_df['Total P&L']))
#page_topic = '' if zero_pnl_num_of_days_trimmed == 0 else ' Trimmed ' + str(zero_pnl_num_of_days_trimmed) + " days of trailing zero pnls"
#tg_pnl_df = tg_pnl_df.iloc[np.trim_zeros(tg_pnl_df['Total P&L']).index].copy()

#if len(tg_pnl_df) == 0:  
    #print("Empty P&L timeseries for tradegroup " + tradar_tg_name + " in fund " + fund)

#tg_pnl_df['Date'].min()
#start_date_yyyymmdd = pd.Timestamp(tg_pnl_df['Date'].min()).to_pydatetime().strftime('%Y%m%d')
#end_date_yyyymmdd = pd.Timestamp(tg_pnl_df['Date'].max()).to_pydatetime().strftime('%Y%m%d')
#tg_bbg_format_tickers =  [x + ' Equity' for x in tg_securities if x not in ['USD']] 
#ticker2hp = bbgclient.bbgclient.get_secid2hp(tg_bbg_format_tickers,'tickers',start_date_yyyymmdd,end_date_yyyymmdd)
#for ticker in ticker2hp:
   # frame = pd.DataFrame(columns = ["Date", ticker])
   # frame['Date'] = ticker2hp[ticker]['Dates']
    #frame[ticker] = ticker2hp[ticker]['Prices']
    #tg_securities_price_dfs.append(frame)
#tg_securities_price_dfs = [(pd.DataFrame(data = zip(ticker2hp[ticker]['Dates'], ticker2hp[ticker]['Prices']), columns=['Date',ticker])).dropna() for ticker in ticker2hp]
#for df in tg_securities_price_dfs: df[df.columns[1]] = df[df.columns[1]].astype(float)

In [26]:
def perf_timeseries_df(pnl_df, capital_df, as_of_date=None, calc_stats=True, periods=('MTD', 'QTD', 'YTD', 'ITD', "1D",
                                                                                      "3D", "5D", "30D")):
    empty_stats_df = pd.DataFrame(columns=["Period", 'P&L($)', 'P&L(bps)', 'ROMC(bps)',
                                            'ANN. VOL', 'CAPITAL($)', 'CAPITAL CHG(%)'])
    empty_stats_df["Period"] = periods
    empty_perf_df = pd.DataFrame(columns=pnl_df.columns)
    empty_res = (empty_perf_df, empty_stats_df) if calc_stats else empty_perf_df
    if len(pnl_df) == 0 or len(pnl_df[pd.isnull(pnl_df['Total P&L'])])>0: return empty_res

    # pnl data present. at least calculate p&l cumsum
    slicer = dfutils.df_slicer(as_of_date)
    empty_stats_df['P&L($)'] = [None if len(sliced_df) == 0 else sliced_df['Total P&L'].sum() 
                                for (period,sliced_df) in zip(periods, [slicer.slice(pnl_df,p) for p in periods])]

    if capital_df is None: return empty_res
    if len(capital_df) == 0 or len(capital_df.columns)<=1: return empty_res
    if len(capital_df[pd.isnull(capital_df[capital_df.columns[1]])])>0: return empty_res

    capital_cln = capital_df.columns[1]
    capital_df_cpy = capital_df.copy() # don't change capital_df. work on its copy
    capital_df_cpy['Capital'] = capital_df_cpy[capital_cln]
    capital_df_cpy['Shifted Capital'] = capital_df_cpy[capital_cln].shift(1)

    df = pd.merge(pnl_df, capital_df_cpy, how = 'left', on = ['Date']).sort_values(by='Date') 
    if pd.isnull(df['Shifted Capital'].iloc[0]): df.loc[0,'Shifted Capital'] = df['Capital'].iloc[0]
    df['Shifted Capital'] = df['Shifted Capital'].apply(lambda x: np.nan if x == 0 else x)
    # if pd.isna(df['Shifted Capital'].iloc[1]):
    #     print('Updating Shifted capital')
    #     df['Shifted Capital'].loc[1] = df['Capital'].loc[1]
    #
    df['Shifted Forward-Filled Capital'] = df['Shifted Capital'].ffill()
    # if pd.isna(df['Shifted Forward-Filled Capital'].iloc[1]):
    #     print('Updating Shifted Forward-filled Capital')
    #     df['Shifted Forward-Filled Capital'].loc[1] = df['Capital'].loc[1]
    try:
        if df['Fund'].iloc[0] == 'MACO':   # Temp Fix for MACO
            df['Shifted Capital'].iloc[1] = df['Capital'].iloc[1]
            df['Shifted Forward-Filled Capital'].iloc[1] = df['Capital'].iloc[1]
        # df.drop(df.index[0], inplace=True)
        #df.to_csv('TEMPDF-aprl3.csv')

    except Exception as err:
        print err
        print 'Fund not found in Dataframe'

    df["P&L bps (ffiled)"] = 1e4*(df["Total P&L"]/df["Shifted Forward-Filled Capital"]).replace([np.inf,
                                                                                                 -np.inf], np.nan)
    df["Cumulative P&L ($)"] = df["Total P&L"].cumsum()
    df["Cumulative P&L bps (ffiled)"] = 1e4*((1.0+(df["P&L bps (ffiled)"].astype(float)/1e4)).cumprod() -1)
    df['Rolling 30D Vol(%)'] = math.sqrt(252)*df['P&L bps (ffiled)'].rolling(window = 30).std()/100.0
    df['Rolling 60D Vol(%)'] = math.sqrt(252)*df['P&L bps (ffiled)'].rolling(window = 60).std()/100.0
    df['Rolling 90D Vol(%)'] = math.sqrt(252)*df['P&L bps (ffiled)'].rolling(window = 90).std()/100.0
    if not calc_stats: return df

    # calc stats
    stats_df = pd.DataFrame(columns=["Period",'P&L($)','P&L(bps)','ROMC(bps)',
                                     'ANN. VOL','CAPITAL($)','CAPITAL CHG(%)'])

    for period in periods:
        p_df = slicer.slice(df, period)

        next_idx = 0 if pd.isnull(stats_df.index.max()) else stats_df.index.max()+1
        if len(p_df) == 0:
            stats_df.loc[next_idx] = [period,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan]
            continue

        rets = p_df["P&L bps (ffiled)"].astype(float)
        vol_pct = math.sqrt(252)*np.std(rets/1e4, ddof=1)*1e2
        cumpnl_usd = p_df['Total P&L'].sum()
        cumret_bps = ((1+rets/1e4).prod() -1)*1e4
        capitals = p_df['Shifted Forward-Filled Capital'].fillna(0).astype(float)
        mean_capital = capitals.mean()
        if mean_capital == 0:
            stats_df.loc[next_idx] = [period,cumpnl_usd,cumret_bps,np.nan,vol_pct,np.nan,np.nan]
            continue
        #mw_rets = (rets/1e4)*(capitals/mean_capital)
        #romwc_bps = ((1+mw_rets).prod() -1)*1e4
        romc_bps = 1e4*(cumpnl_usd/mean_capital)
        capital_usd = capitals.iloc[-1]
        capital_chg_pct = 1e2*((capitals.iloc[-1]/capitals.iloc[0])-1.0) if capitals.iloc[0] != 0 else np.nan
        stats_df.loc[next_idx] = [period, cumpnl_usd, cumret_bps, romc_bps, vol_pct,
                                  capital_usd, capital_chg_pct]
    return df, stats_df

In [27]:
perf_ts_df = perf_timeseries_df(tg_pnl_df, fund_nav_df, calc_stats= False)
pnl_dfs  = []
tg_return_on_nav_dfs = []
options_pnl_df = securities_pnl_df[securities_pnl_df['SecType'] 
                                   == 'Option'][['Date','Total P&L']].groupby('Date').sum().reset_index().copy()
options_pnl_df = options_pnl_df.iloc[np.trim_zeros(options_pnl_df['Total P&L']).index].copy()

if len(options_pnl_df) > 0:
    options_perf_ts_df = perf_timeseries_df(options_pnl_df, fund_nav_df,calc_stats=False)
    options_perf_ts_df = options_perf_ts_df.rename(columns={'Cumulative P&L bps (ffiled)':'Options'}).sort_values(by='Date')
    tg_return_on_nav_dfs.append(options_perf_ts_df[['Date','Options']])
    pnl_dfs.append(options_perf_ts_df[['Date','Cumulative P&L ($)']]
                   .rename(columns={'Cumulative P&L ($)':'Options'}).sort_values(by='Date'))

tg_securities = securities_pnl_df[securities_pnl_df['SecType'] != 'Option']['Ticker'].unique()
tg_return_on_nav_dfs_dict = {}
for ticker in tg_securities:
    tkr_pnl_df = securities_pnl_df[securities_pnl_df['Ticker'] == ticker].copy()
    tkr_pnl_df = tkr_pnl_df.loc[np.trim_zeros(tkr_pnl_df['Total P&L']).index].copy()
    if len(tkr_pnl_df) == 0: continue
    tkr_perf_ts_df = perf_timeseries_df(tkr_pnl_df, fund_nav_df, calc_stats=False)
    tkr_perf_ts_df = tkr_perf_ts_df.rename(columns={'Cumulative P&L bps (ffiled)': ticker}).sort_values(by = 'Date')
    tg_return_on_nav_dfs.append(tkr_perf_ts_df[['Date',ticker]])
    pnl_dfs.append(tkr_perf_ts_df[['Date',"Cumulative P&L ($)"]]
                   .rename(columns={'Cumulative P&L ($)':ticker}).sort_values(by = 'Date'))

pnl_dfs.append(perf_ts_df[["Date", "Total P&L"]].rename(columns={'Total P&L':'Daily P&L($)'}))
pnl_dfs.append(perf_ts_df[["Date", "Cumulative P&L ($)"]].rename(columns={'Cumulative P&L ($)':tradar_tg_name}))

tg_perf_ts_df = perf_ts_df.rename(columns={'Cumulative P&L bps (ffiled)':tradar_tg_name})
tg_return_on_nav_dfs.append(tg_perf_ts_df[['Date',tradar_tg_name]])

tg_return_on_nav_dict = {}
for i in range(0, len(tg_return_on_nav_dfs)):
    col_names = tg_return_on_nav_dfs[i].columns
    series = pd.Series(tg_return_on_nav_dfs[i][col_names[1]].values,
                       index=tg_return_on_nav_dfs[i]['Date'].dt.strftime('%Y-%m-%d')).to_dict()
    tg_return_on_nav_dict[col_names[1]] = series
    
if 'Options' in tg_return_on_nav_dict.keys():
    options_return_on_nav_dict = tg_return_on_nav_dict["Options"]
else: options_return_on_nav_dict = None
if tradar_tg_name in tg_return_on_nav_dict.keys():
    tradegroup_return_on_nav_dict = tg_return_on_nav_dict[tradar_tg_name]
else: tradegroup_return_on_nav_dict = None
if tg_securities[0] in tg_return_on_nav_dict.keys():
    security1_return_on_nav_dict = tg_return_on_nav_dict[tg_securities[0]]
else: security1_return_on_nav_dict = None
if tg_securities[1] in tg_return_on_nav_dict.keys():
    security2_return_on_nav_dict = tg_return_on_nav_dict[tg_securities[1]]
else: security2_return_on_nav_dict = None

pnl_performance_dollar_dict = {}
for i in range(0, len(pnl_dfs)):
    col_names = pnl_dfs[i].columns
    series = pd.Series(pnl_dfs[i][col_names[1]].values, index = pnl_dfs[i]['Date'].dt.strftime('%Y-%m-%d')).to_dict()
    pnl_performance_dollar_dict[col_names[1]] = series
    
if 'Options' in pnl_performance_dollar_dict.keys():
    options_pnl_performance_dict = pnl_performance_dollar_dict["Options"]
else: options_pnl_performance_dict = None
if tradar_tg_name in pnl_performance_dollar_dict.keys():
    tradegroup_pnl_performance_dict = pnl_performance_dollar_dict[tradar_tg_name]
else: tradegroup_pnl_performance_dict = None
if tg_securities[0] in pnl_performance_dollar_dict.keys():
    security1_pnl_performance_dict = pnl_performance_dollar_dict[tg_securities[0]]
else: security1_pnl_performance_dict = None
if tg_securities[1] in pnl_performance_dollar_dict.keys():
    security2_pnl_performance_dict = pnl_performance_dollar_dict[tg_securities[1]]
else: security2_pnl_performance_dict = None
if 'Daily P&L($)' in pnl_performance_dollar_dict.keys():
    daily_pnl_performance_dict = pnl_performance_dollar_dict['Daily P&L($)']
else: daily_pnl_performance_dict = None


'Fund'
Fund not found in Dataframe


In [28]:
def mktval_df(pcvh_df):
    float_cols = ['Alpha Exposure','AlphaHedge Exposure','Hedge Exposure',
                  'Alpha NetMktVal', 'AlphaHedge NetMktVal','Hedge NetMktVal',
                  'Alpha GrossMktVal','AlphaHedge GrossMktVal','Hedge GrossMktVal',
                  'Alpha GrossExp','AlphaHedge GrossExp','Hedge GrossExp']
    calc_cols = ['Bet Exposure','Bet NetMktVal','Bet GrossMktVal','Total NetExposure',
                 'Total NetMktVal','Total GrossMktVal',]
 
    if len(pcvh_df) == 0: return pd.DataFrame(columns=["Date"]+float_cols+calc_cols)

    alpha_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Alpha']
    alphahedge_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Alpha Hedge']
    hedge_pcvh = pcvh_df[pcvh_df['AlphaHedge'] == 'Hedge']

    alpha_net_exp = alpha_pcvh[['Date','Exposure_Net']].groupby('Date').sum().reset_index().rename(columns={'index':'Date', 'Exposure_Net':'Alpha Exposure'})
    alpha_net_mv = alpha_pcvh[['Date','NetMktVal']].groupby('Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'Alpha NetMktVal'})
    alpha_gross_mv = alpha_pcvh[['Date','NetMktVal']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date', 'NetMktVal':'Alpha GrossMktVal'})
    alpha_gross_exp = alpha_pcvh[['Date','Exposure_Net']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date', 'Exposure_Net':'Alpha GrossExp'})

    alphahedge_net_exp = alphahedge_pcvh[['Date','Exposure_Net']].groupby('Date').sum().reset_index().rename(columns={'index':'Date', 'Exposure_Net':'AlphaHedge Exposure'})
    alphahedge_net_mv = alphahedge_pcvh[['Date','NetMktVal']].groupby('Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'AlphaHedge NetMktVal'})
    alphahedge_gross_mv = alphahedge_pcvh[['Date','NetMktVal']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date', 'NetMktVal':'AlphaHedge GrossMktVal'})
    alphahedge_gross_exp = alphahedge_pcvh[['Date','Exposure_Net']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date', 'Exposure_Net':'AlphaHedge GrossExp'})

    hedge_net_exp = hedge_pcvh[['Date','Exposure_Net']].groupby('Date').sum().reset_index().rename(columns={'index':'Date','Exposure_Net':'Hedge Exposure'})
    hedge_net_mv = hedge_pcvh[['Date','NetMktVal']].groupby('Date').sum().reset_index().rename(columns={'index':'Date', 'NetMktVal':'Hedge NetMktVal'})
    hedge_gross_mv = hedge_pcvh[['Date','NetMktVal']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date', 'NetMktVal':'Hedge GrossMktVal'})
    hedge_gross_exp = hedge_pcvh[['Date','Exposure_Net']].groupby('Date').agg(lambda x: sum(abs(x))).reset_index().rename(columns={'index':'Date', 'Exposure_Net':'Hedge GrossExp'})

    mktval_df=pd.merge(alpha_net_exp, alphahedge_net_exp, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, hedge_net_exp, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, alpha_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, alphahedge_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, hedge_net_mv, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, alpha_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, alphahedge_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, hedge_gross_mv, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, alpha_gross_exp, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, alphahedge_gross_exp, how='outer', on=['Date']).fillna(0)
    mktval_df=pd.merge(mktval_df, hedge_gross_exp, how='outer', on=['Date']).fillna(0)

    mktval_df[float_cols] = mktval_df[float_cols].astype(float)
    mktval_df['Bet Exposure'] = mktval_df['Alpha Exposure']+mktval_df['AlphaHedge Exposure']
    mktval_df['Bet NetMktVal'] = mktval_df['Alpha NetMktVal']+mktval_df['AlphaHedge NetMktVal']
    mktval_df['Bet GrossMktVal'] = mktval_df['Alpha GrossMktVal']+mktval_df['AlphaHedge GrossMktVal']
    mktval_df['Total NetExposure'] = mktval_df['Alpha Exposure']+mktval_df['AlphaHedge Exposure']+mktval_df['Hedge Exposure']
    mktval_df['Total GrossExposure'] = mktval_df['Alpha GrossExp']+mktval_df['AlphaHedge GrossExp']+mktval_df['Hedge GrossExp']
    mktval_df['Total NetMktVal'] = mktval_df['Alpha NetMktVal']+mktval_df['AlphaHedge NetMktVal']+mktval_df['Hedge NetMktVal']
    mktval_df['Total GrossMktVal'] = mktval_df['Alpha GrossMktVal']+mktval_df['AlphaHedge GrossMktVal']+mktval_df['Hedge GrossMktVal']


    return mktval_df.sort_values(by='Date')


In [29]:
###NEED to FIX
if len(tg_pcvh) > 0:
    tg_capital_df = mktval_df(tg_pcvh)
    roc_df = perf_timeseries_df(tg_pnl_df, tg_capital_df[['Date','Bet GrossMktVal']], calc_stats=False)
    roc_df['Cumulative P&L (%)'] = roc_df['Cumulative P&L bps (ffiled)'].apply(lambda x: x/100.0)
    roc_df[['Date','Cumulative P&L (%)']].dropna()
    perf_ts_df['Rolling 30D P&L annualized vol(%)'] = math.sqrt(252)*(roc_df['P&L bps (ffiled)']
                                                                      /100.0).rolling(window=30).std()
    if len(perf_ts_df['Rolling 30D P&L annualized vol(%)'].dropna())>0:
        perf_ts_df[["Date", "Rolling 30D P&L annualized vol(%)"]].dropna()

pnl_performance_over_cap_dict = pd.Series(roc_df['Cumulative P&L (%)'].values,
                                          index=roc_df.Date.dt.strftime('%Y-%m-%d')).to_dict()
rolling_pnl_30D_vol_dict = pd.Series(perf_ts_df["Rolling 30D P&L annualized vol(%)"].values,
                                     index=perf_ts_df.Date.dt.strftime('%Y-%m-%d')).to_dict()

In [30]:
def get_NAV_df2(fund_code=None):  # didn't find required fields
    code2name = {"CAM": "Columbia",
                "LG": "Litman Gregory",
                "TACO": "The Arbitrage Credit Opportunities Fund",
                "AED": "The Arbitrage Event-Driven Fund",
                "ARB": "The Arbitrage Fund",
                "TAQ": "The Arbitrage Tactical Equity Fund",
                "TAF": "TransAmerica",
                "WIC": "WIC Arbitrage Partners",
                "WED": "Water Island Event-Driven Fund",
                "LEV": "Water Island Capital Lev Arb Fund",
                "MACO": "WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL",
                "MALT": "Morningstar Alternatives Fund"
                }

    if fund_code not in code2name: return pd.DataFrame(columns=['Date', 'NAV'])

    query = "SELECT " \
            "[Date], " \
            "NAV " \
            "FROM PnLAppDb.pnl.DailyNAV AS A " \
            "INNER JOIN PnLAppDb.dbo.Funds AS B ON A.FundId = B.FundID " \
            "WHERE B.FundName = '" + code2name[fund_code] + "' " \
            "ORDER BY [DATE]"

    results = optimized_execute(query)
    df = pd.DataFrame(results, columns=["Date", "NAV"])
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df.index.name = 'Date'
    df = df.sort_index()
    df['NAV'] = df['NAV'].astype(float)
    # special NAV treatment for northpoint mutual fund issue
    df['NAV'] = list(df['NAV'][1:]) + [None]
    idx = pd.date_range(df['Date'].min(), df['Date'].max())
    df = df.reindex(idx, fill_value=np.nan)
    del df['Date']
    df.reset_index(level=0, inplace=True)
    df.rename(columns={"index": "Date"}, inplace=True)
    df.index.name = 'Date'
    df.ffill(inplace=True)
    df.set_index(pd.DatetimeIndex(df['Date']), inplace=True)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    df = df.reset_index(level=0, drop=True).reset_index()
    del df['index']
    return df[df['Date'] <= (datetime.datetime.today() - datetime.timedelta(days=1))]


In [31]:
def get_tradegroups_snapshot():
    # region query
    query = "SELECT Fund, Sleeve, TradeGroup, Analyst, LongShort, InceptionDate, EndDate, Status," \
            "`Metrics in Bet JSON`,`Metrics in Bet notes JSON`,`Metrics in NAV JSON`,`Metrics in NAV notes JSON` " \
            "FROM " + DB_NAME + ".tradegroups_snapshot2"
    # endregion

    res = wic_optimized_execute(query)
    cols = ['Fund', 'Sleeve', 'TradeGroup', 'Analyst', 'LongShort', 'InceptionDate', 'EndDate', 'Status',
            'Metrics in Bet JSON', 'Metrics in Bet notes JSON', 'Metrics in NAV JSON', 'Metrics in NAV notes JSON']
    return pd.DataFrame(list(res), columns=cols)

In [32]:
def df2row(pivot_col, df):
    dfcols = [c for c in df.columns if c != pivot_col]
    cols = [colname+'|'+period for (colname, period) in itertools.product(dfcols, df[pivot_col])]
    cols2vals = {c: None for c in cols}

    for idx in df.index:
        row = df.iloc[idx]
        pivot = row[pivot_col]
        for cln in dfcols:
            key = cln+'|'+pivot
            cols2vals[key] = row[cln]
    res = pd.Series()
    for cln in cols:
        res[cln] = cols2vals[cln]
    return res

In [33]:
def json2row(json):
    df = pd.read_json(json)
    return df2row('Period', df)

In [34]:
def get_tradegroups_attribution_to_fund_nav():
    today = datetime.datetime.today()

    df = get_tradegroups_snapshot()

    df['EndDate'] = df['EndDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%m/%d/%y'))
    df['InceptionDate'] = df['InceptionDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%m/%d/%y'))
    #Do not want tradegroups closed before the start of the current year
    #df_reduced = df[df["EndDate"].dt.year >= today.year] #Takes out deals like cash that don't have date- wrong

    metrics2include = [('P&L(bps)', 'ITD'), ('P&L($)', 'ITD'),
                       ('P&L(bps)', 'YTD'), ('P&L($)', 'YTD'),
                       ('P&L(bps)', 'QTD'), ('P&L($)', 'QTD'),
                       ('P&L(bps)', 'MTD'), ('P&L($)', 'MTD'),
                       ('P&L(bps)', '30D'), ('P&L($)', '30D'),
                       ('P&L(bps)', '5D'), ('P&L($)', '5D'),
                       ('P&L(bps)', '1D'), ('P&L($)', '1D')]

    metric2display_name = {'P&L(bps)': '', 'P&L($)': ''}
    metric2unit = {'P&L(bps)': 'bps', 'P&L($)': '$'}

    # unjsonify metrics, and append columns
    metrics_df = pd.DataFrame([json2row(json) for json in df['Metrics in NAV JSON']])
    metrics_df.index = df.index

    #cln2decimal_pts = {}
    #colnames_to_sum = []
    #display_columns = []
    for (metric, period) in metrics2include:
        unit = metric2unit[metric]
        disp_name = metric2display_name[metric]
        display_colname = disp_name + ' ' + period + '(' + unit + ')'
        df[display_colname] = metrics_df[metric + '|' + period]
        #if unit == '$':
            #cln2decimal_pts[display_colname] = 0
            #colnames_to_sum.append(display_colname)
        #if unit == 'bps':
            #cln2decimal_pts[display_colname] = 1
            #colnames_to_sum.append(display_colname)
        #if unit == '%':
            #cln2decimal_pts[display_colname] = 2
        #display_columns.append(display_colname)

    del df['Metrics in NAV JSON']; del df['Metrics in NAV notes JSON'];
    del df['Metrics in Bet JSON']; del df['Metrics in Bet notes JSON'];
    del df['Analyst']

    sleeve2code = {'Merger Arbitrage': 'M&A', 'Equity Special Situations': 'ESS',
                   'Opportunistic' : 'OPP', 'Forwards':'FWD', 'Credit Opportunities':'CREDIT'}

    df['Sleeve'] = df['Sleeve'].apply(lambda x: sleeve2code[x] if x in sleeve2code else x)
    df = df[(~pd.isnull(df[' YTD($)']))]  # don't show null ytds. i.e. tradegroups closed before year started
    df['Date'] = today.strftime('%Y-%m-%d')

    base_cols = ['Date', 'Fund', 'Sleeve', 'TradeGroup', 'LongShort', 'InceptionDate', 'EndDate', 'Status']
    bps_cols = [' ITD(bps)', ' YTD(bps)', ' QTD(bps)', ' MTD(bps)', ' 30D(bps)', ' 5D(bps)', ' 1D(bps)']  
    dollar_cols = [' ITD($)', ' YTD($)', ' QTD($)', ' MTD($)', ' 30D($)', ' 5D($)', ' 1D($)']
    bps_df = df[base_cols+bps_cols].sort_values(by=' YTD(bps)')
    bps_df.rename(columns={' ITD(bps)': 'ITD(bps)', ' YTD(bps)': 'YTD(bps)', ' QTD(bps)': 'QTD(bps)',
                           ' MTD(bps)': 'MTD(bps)', ' 30D(bps)': '30D(bps)', ' 5D(bps)': '5D(bps)',
                           ' 1D(bps)': '1D(bps)'}, inplace=True)
    dollar_df = df[base_cols+dollar_cols].sort_values(by=' YTD($)')
    dollar_df.rename(columns={' ITD($)': 'ITD($)', ' YTD($)': 'YTD($)', ' QTD($)': 'QTD($)',
                              ' MTD($)': 'MTD($)', ' 30D($)': '30D($)', ' 5D($)': '5D($)',
                              ' 1D($)': '1D($)'}, inplace=True)

    return bps_df, dollar_df

In [35]:
#TradeGroup Attribution to Fund NAV Page
x = get_tradegroups_attribution_to_fund_nav() 
#Produces two dataframes: one for bps and one for dollar
unique_tradegroups = x[0]['TradeGroup'].unique()
limit_to_tradegroups = list(unique_tradegroups)
unique_funds = x[0]['Fund'].unique()
limit_to_funds = list(unique_funds)

In [60]:
x[0]

,Date,Fund,Sleeve,TradeGroup,LongShort,InceptionDate,EndDate,Status,ITD(bps),YTD(bps),QTD(bps),MTD(bps),30D(bps),5D(bps),1D(bps)
4812,2019-03-21 11:26:49.491,WED,OPP,NIHD CONV - TRADE,Long,04/18/18,03/20/19,ACTIVE,-1156.110785,-971.867533,-971.867533,-874.765746,-1153.601735,-678.894286,-68.330146
4590,2019-03-21 11:26:49.491,WED,OPP,ARLO CONV - TRADE,Long,08/03/18,03/20/19,ACTIVE,-739.455199,-396.389146,-396.389146,-17.163114,-305.154858,-18.997854,-4.002477
4896,2019-03-21 11:26:49.491,WED,OPP,SPY MARKET HEDGE,Short,04/19/17,03/20/19,ACTIVE,-606.355165,-271.533274,-271.533274,-53.345608,-88.327174,-57.841376,0.812786
4726,2019-03-21 11:26:49.491,WED,OPP,GSAT CONV - TRADE,Long,05/12/17,03/20/19,ACTIVE,-25.428592,-69.291092,-69.291092,-14.805129,-55.699607,7.948819,-3.065582
4646,2019-03-21 11:26:49.491,WED,OPP,CLD BONDS CONV - TRADE,Long,03/08/19,03/20/19,ACTIVE,-45.737096,-45.737096,-45.737096,-45.737096,-45.737096,-49.201618,-15.167962
2383,2019-03-21 11:26:49.491,LEV,None,CASH,NA,None,None,None,-209.300755,-34.731836,-34.731836,NaN,-11.017434,NaN,NaN
3798,2019-03-21 11:26:49.491,TACO,CREDIT,CREDIT MACRO HEDGES,Short,01/04/16,03/20/19,ACTIVE,-69.851906,-30.598103,-30.598103,-3.617439,-7.389573,-1.845579,-1.383426
4127,2019-03-21 11:26:49.491,TAQ,ESS,ARLO R/R,Long,01/02/19,03/20/19,ACTIVE,-29.511738,-29.511738,-29.511738,-0.229085,-3.363948,-0.211129,-0.057439
4795,2019-03-21 11:26:49.491,WED,ESS,MFGP R/R SHORT,Short,08/23/17,02/21/19,CLOSED,-45.746114,-29.181654,-29.181654,NaN,-24.876106,NaN,NaN
4456,2019-03-21 11:26:49.491,TAQ,ESS,SERV R/R SHORT 2,Short,02/07/19,03/20/19,ACTIVE,-23.893793,-23.893793,-23.893793,-7.277307,-23.893793,2.693317,0.970794


In [44]:
'''def get_securities_pnl_by_tradegroup_and_fund(limit_to_tradegroups=[], limit_to_funds=[],
                                              is_tg_names_in_tradar_format=False, start_date_yyyymmdd=None,
                                              end_date_yyyymmdd=None, rollup_pnl=False):
    if start_date_yyyymmdd is None: start_date_yyyymmdd = '20170101'
    if end_date_yyyymmdd is None:
        now = datetime.datetime.now()
        slicer = dfutils.df_slicer()
        end_date = slicer.prev_n_business_days(1, now)
        end_date_yyyymmdd = end_date.strftime('%Y%m%d')

    tg_filter_clause = ""
    if len([tg for tg in limit_to_tradegroups if tg is not None]) > 0:
        tg_filter_clause = " AND B.strat IN (" + ",".join([("'" + tg + "'")
                                                           for tg in limit_to_tradegroups 
                                                           if tg is not None]) + ") "
    if is_tg_names_in_tradar_format is True: 
        tg_filter_clause = tg_filter_clause.replace('B.strat', 'SUBSTRING(B.strat,0,21)')
        
    if len([fund for fund in limit_to_funds if fund is not None]) > 0:
        fund_code_filter_clause = " AND F.fund IN (" + ",".join([("'" + fund + "'")
                                                                 for fund in limit_to_funds
                                                                 if fund is not None]) + ") "
        
    # region rollup query
    rollup_sql_query = "DECLARE @Holidays TABLE ([Date] DATE); " \
                        "DECLARE @StartDt INT; " \
                        "DECLARE @EndDt INT; " \
                        "INSERT INTO @Holidays VALUES " \
                        "('2015-01-01'), " \
                        "('2015-01-19'), " \
                        "('2015-02-16'), " \
                        "('2015-04-03'), " \
                        "('2015-05-25'), " \
                        "('2015-07-03'), " \
                        "('2015-09-07'), " \
                        "('2015-10-12'), " \
                        "('2015-11-11'), " \
                        "('2015-11-26'), " \
                        "('2015-12-25'), " \
                        "('2016-01-01'), " \
                        "('2016-01-18'), " \
                        "('2016-02-15'), " \
                        "('2016-03-25'), " \
                        "('2016-05-30'), " \
                        "('2016-07-04'), " \
                        "('2016-09-05'), " \
                        "('2016-11-24'), " \
                        "('2016-12-25'), " \
                        "('2017-01-02'), " \
                        "('2017-01-16'), " \
                        "('2017-02-20'), " \
                        "('2017-04-14'), " \
                        "('2017-05-29'), " \
                        "('2017-07-04'), " \
                        "('2017-09-04'), " \
                        "('2017-10-09'), " \
                        "('2017-11-23'), " \
                        "('2017-12-25'); " \
                        "SET @StartDt = " + start_date_yyyymmdd + "; " \
                        "SET @EndDt = " + end_date_yyyymmdd + "; " \
                        "WITH X AS( " \
                        "SELECT " \
                        "CAST(CONVERT(varchar(8), A.timeKey) AS DATE) as [Date], " \
                        "F.fund, " \
                        "B.strat AS TradeGroup, " \
                        "S.ticker, " \
                        "ST.groupingName as sectype, " \
                        "SUM(pnlFC) as PNL " \
                        "FROM performanceAttributionFact AS A " \
                        "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
                        "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
                        "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
                        "INNER JOIN SecType AS ST ON ST.sectype = S.sectype " \
                        "WHERE A.timeKey >= @StartDt and A.timeKey <= @EndDt " \
                        + tg_filter_clause + \
                        fund_code_filter_clause + \
                        "GROUP BY A.timeKey, B.strat, F.fund, S.ticker, ST.groupingName " \
                        ") " \
                        ", PNL AS ( " \
                        "SELECT " \
                        "X.[Date],  " \
                        "CASE " \
                        "WHEN DATEPART(dw,X.[Date]) IN (7,1) THEN 'WEEKEND' " \
                        "WHEN X.[Date] IN (SELECT * FROM @Holidays) THEN 'HOLIDAY' " \
                        "ELSE 'BUSINESS DAY' END AS [WeekDay], " \
                        "X.Fund, " \
                        "X.TradeGroup, " \
                        "X.Ticker, " \
                        "X.sectype, " \
                        "X.PNL " \
                        "FROM X " \
                        "), " \
                        "Position2EndDate AS ( " \
                        "SELECT " \
                        "X.fund, " \
                        "X.TradeGroup, " \
                        "X.ticker, " \
                        "X.sectype, " \
                        "MAX(X.[Date]) AS MaxDate, " \
                        "CASE DATEPART(dw,MAX(X.[Date])) " \
                        "WHEN 6 THEN DATEADD(DAY,3,MAX(X.[Date])) " \
                        "WHEN 7 THEN DATEADD(DAY,2,MAX(X.[Date])) " \
                        "ELSE DATEADD(DAY,1,MAX(X.[Date])) END AS NextBusinessDay " \
                        "FROM PNL AS X " \
                        "GROUP BY X.fund, X.TradeGroup, X.ticker, X.sectype " \
                        "), " \
                        "Position2EndDate_BD AS ( " \
                        "SELECT " \
                        "X.fund, " \
                        "X.TradeGroup, " \
                        "X.ticker, " \
                        "X.sectype, " \
                        "X.MaxDate, " \
                        "CASE " \
                        "WHEN NextBusinessDay IN (SELECT * FROM @Holidays) THEN " \
                        "CASE DATEPART(dw,NextBusinessDay) " \
                        "WHEN 6 THEN DATEADD(DAY,3,NextBusinessDay) " \
                        "ELSE DATEADD(DAY,1,NextBusinessDay) END " \
                        "ELSE NextBusinessDay " \
                        "END AS NextBusinessDay_Holiday_Proof " \
                        "FROM Position2EndDate AS X " \
                        "), " \
                        "PNL_DT AS ( " \
                        "SELECT " \
                        "A1.[Date], " \
                        "A1.[WeekDay],  " \
                        "A1.fund, " \
                        "A1.TradeGroup, " \
                        "A1.ticker, " \
                        "A1.sectype, " \
                        "A1.PNL, " \
                        "MAX(A2.[Date]) AS [Last Business Day], " \
                        "DATEDIFF(DAY,MAX(A2.[Date]),A1.[Date]) AS [Last BD days diff] " \
                        "FROM PNL AS A1 " \
                        "LEFT OUTER JOIN PNL AS A2 ON A1.fund = A2.fund and A1.TradeGroup = A2.TradeGroup " \
                        "AND A1.ticker = A2.ticker AND A1.sectype = A2.sectype AND A2.[Date] < A1.[Date] " \
                        "AND A2.[WeekDay] = 'BUSINESS DAY' " \
                        "GROUP BY A1.[Date],A1.[WeekDay],A1.fund, A1.TradeGroup, A1.ticker, A1.sectype, A1.PNL " \
                        "), " \
                        "T AS ( " \
                        "SELECT " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN B.NextBusinessDay_Holiday_Proof ELSE A1.[Date] END AS [Date], " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN 'BUSINESS DAY' ELSE A1.[WeekDay] END AS [WeekDay], " \
                        "A1.fund, " \
                        "A1.TradeGroup, " \
                        "A1.ticker, " \
                        "A1.sectype, " \
                        "A1.[Last Business Day], " \
                        "A1.PNL, " \
                        "ISNULL(SUM(A2.PNL),0) AS ROLLED_UP_PNL, " \
                        "A1.PNL + ISNULL(SUM(A2.PNL),0) AS TOT_PNL,  " \
                        "CASE WHEN A1.[Date] = B.MaxDate AND A1.[WeekDay] <> 'BUSINESS DAY' " \
                        "THEN 'Closed on non-BD. Shifted to next BD' ELSE NULL END AS 'Notes' " \
                        "FROM PNL_DT AS A1 " \
                        "LEFT OUTER JOIN PNL_DT AS A2 " \
                        "ON A1.fund = A2.fund AND A1.TradeGroup = A2.TradeGroup AND A1.ticker = A2.ticker " \
                        "AND A1.sectype = A2.sectype AND " \
                        "(A2.[Date] < A1.[Date] AND (A2.[Date] > A1.[Last Business Day] " \
                        "OR A1.[Last Business Day] IS NULL)) " \
                        "AND (A1.[Last BD days diff] <= 4 OR A1.[Last BD days diff] IS NULL) " \
                        "INNER JOIN Position2EndDate_BD AS B ON A1.fund = B.fund AND A1.TradeGroup = B.TradeGroup " \
                        "AND A1.ticker = B.ticker AND A1.sectype = B.sectype " \
                        "WHERE A1.[WeekDay] = 'BUSINESS DAY' OR A1.[Date] = B.MaxDate " \
                        "GROUP BY A1.[Date],A1.[WeekDay], A1.fund,A1.TradeGroup, A1.ticker, " \
                        "A1.sectype, A1.[Last Business Day],A1.[Last BD days diff],A1.PNL, B.MaxDate, " \
                        "B.NextBusinessDay_Holiday_Proof " \
                        ") " \
                        "SELECT " \
                        "T.[Date],  " \
                        "T.fund, " \
                        "T.TradeGroup, " \
                        "T.ticker, " \
                        "T.sectype, " \
                        "T.TOT_PNL " \
                        "FROM T " \
                        "ORDER BY T.fund, T.TradeGroup, T.ticker,T.sectype, T.[Date] "
        
    # endregion
    # region non-rollup query
    # don't take today's pnl - tradar's garbage data
    non_rollup_sql_query = \
        "SELECT " \
        "CAST(CONVERT(VARCHAR(8), A.timeKey) AS DATE) AS [DATE], " \
        "F.fund, " \
        "B.strat AS TradeGroup, " \
        "S.ticker, " \
        "ST.groupingName AS sectype, " \
        "SUM(pnlFC) AS PNL " \
        "FROM performanceAttributionFact AS A " \
        "INNER JOIN TradeStrat AS B ON A.stratKey = B.stratId " \
        "INNER JOIN TradeFund AS F ON A.fundKey = F.fundId " \
        "INNER JOIN Sec AS S ON S.secId = A.secIdKey " \
        "INNER JOIN SecType AS ST ON ST.sectype = S.sectype " \
        "WHERE A.timeKey >= " + start_date_yyyymmdd + " AND A.timeKey <= " \
        + end_date_yyyymmdd + \
        tg_filter_clause + \
        fund_code_filter_clause + \
        "GROUP BY A.timeKey, B.strat, F.fund,S.ticker,ST.groupingName " \
        "ORDER BY F.fund, B.strat,S.ticker,A.timeKey "
    # endregion
    query = rollup_sql_query if rollup_pnl else non_rollup_sql_query
    cols = ['Date', 'Fund', 'TradeGroup', 'Ticker', 'SecType', 'Total P&L']
    try:
        query_result = tradar_optimized_execute(query)
    except:
        return pd.DataFrame(columns=cols)

    df = pd.DataFrame(query_result, columns=cols)
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(x))
    df['Total P&L'] = df['Total P&L'].astype(float)
    df['Cumulative Total P&L'] = None
    return df
    '''

'def get_securities_pnl_by_tradegroup_and_fund(limit_to_tradegroups=[], limit_to_funds=[],\n                                              is_tg_names_in_tradar_format=False, start_date_yyyymmdd=None,\n                                              end_date_yyyymmdd=None, rollup_pnl=False):\n    if start_date_yyyymmdd is None: start_date_yyyymmdd = \'20170101\'\n    if end_date_yyyymmdd is None:\n        now = datetime.datetime.now()\n        slicer = dfutils.df_slicer()\n        end_date = slicer.prev_n_business_days(1, now)\n        end_date_yyyymmdd = end_date.strftime(\'%Y%m%d\')\n\n    tg_filter_clause = ""\n    if len([tg for tg in limit_to_tradegroups if tg is not None]) > 0:\n        tg_filter_clause = " AND B.strat IN (" + ",".join([("\'" + tg + "\'")\n                                                           for tg in limit_to_tradegroups \n                                                           if tg is not None]) + ") "\n    if is_tg_names_in_tradar_format is True: \n    

In [45]:
#get_securities_pnl_by_tradegroup_and_fund(limit_to_tradegroups=limit_to_tradegroups, limit_to_funds=limit_to_funds,
 #                                             is_tg_names_in_tradar_format=True, rollup_pnl=True)

In [35]:
x[1]

,Date,Fund,Sleeve,TradeGroup,LongShort,InceptionDate,EndDate,Status,ITD($),YTD($),QTD($),MTD($),30D($),5D($),1D($)
1094,2019-03-21 09:29:51.700,ARB,M&A,GG - NEM,Long,01/14/19,03/20/19,ACTIVE,-1.031702e+06,-1.031702e+06,-1.031702e+06,1.005004e+06,-1.062602e+06,33321.036700,-12645.900000
4812,2019-03-21 09:29:51.700,WED,OPP,NIHD CONV - TRADE,Long,04/18/18,03/20/19,ACTIVE,-8.067661e+05,-7.217765e+05,-7.217765e+05,-6.699357e+05,-9.042629e+05,-510081.892300,-50552.480000
1134,2019-03-21 09:29:51.700,ARB,M&A,IDTI - RENESAS,Long,09/11/18,03/20/19,ACTIVE,5.245788e+04,-4.916999e+05,-4.916999e+05,-1.506053e+05,-6.885707e+05,-166744.634000,152811.470000
2835,2019-03-21 09:29:51.700,LG,M&A,GG - NEM,Long,01/14/19,03/20/19,ACTIVE,-4.163826e+05,-4.163826e+05,-4.163826e+05,4.114298e+05,-4.261979e+05,19049.395800,-4486.960000
1263,2019-03-21 09:29:51.700,ARB,M&A,NFX - ECA REVERSAL,Short,11/07/18,02/25/19,CLOSED,-5.701438e+05,-3.968777e+05,-3.968777e+05,NaN,-1.444402e+04,NaN,NaN
2601,2019-03-21 09:29:51.700,LG,ESS,ARLO R/R,Long,01/02/19,03/20/19,ACTIVE,-3.816280e+05,-3.816280e+05,-3.816280e+05,-9.907200e+03,-1.436544e+05,-9081.600000,-2476.800000
2763,2019-03-21 09:29:51.700,LG,ESS,DOV R/R SHORT,Short,12/29/17,03/20/19,ACTIVE,-2.002635e+05,-3.618590e+05,-3.618590e+05,-1.092204e+05,-1.018541e+05,-18459.979591,17729.330000
2994,2019-03-21 09:29:51.700,LG,CREDIT,MEGCN 6.50 1/15/25 MERGER BONDS,Long,01/08/19,01/18/19,CLOSED,-3.499283e+05,-3.499283e+05,-3.499283e+05,NaN,NaN,NaN,NaN
3169,2019-03-21 09:29:51.700,LG,ESS,SERV R/R SHORT 2,Short,02/07/19,03/20/19,ACTIVE,-3.495585e+05,-3.495585e+05,-3.495585e+05,-1.031119e+05,-3.495585e+05,40725.000000,14661.000000
4590,2019-03-21 09:29:51.700,WED,OPP,ARLO CONV - TRADE,Long,08/03/18,03/20/19,ACTIVE,-5.853187e+05,-2.925193e+05,-2.925193e+05,-1.187289e+04,-2.173821e+05,-14051.140000,-2961.140000


In [36]:
def get_fund_name2code():
    ''' Fund names '''
    return {"Columbia": "CAM",
            "Litman Gregory": "LG",
            "The Arbitrage Credit Opportunities Fund": "TACO",
            "The Arbitrage Event-Driven Fund": "AED",
            "The Arbitrage Fund": "ARB",
            "The Arbitrage Tactical Equity Fund": "TAQ",
            "TransAmerica": "TAF",
            "WIC Arbitrage Partners": "WIC",
            "Water Island Event-Driven Fund": "WED",
            "Water Island Capital Lev Arb Fund": "LEV",
            "WATER ISLAND MERGER ARBITRAGE INSTITUTIONAL": "MACO",
            "Morningstar Alternatives Fund": "MALT"
           }

def get_fund_code2name():
    return {v: k for (k, v) in get_fund_name2code().iteritems()}

In [37]:
def get_tradegroup_performance_over_own_capital():
    try:
        today = datetime.datetime.today()
        
        df = get_tradegroups_snapshot()

        df['EndDate'] = df['EndDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%m/%d/%y'))
        df['InceptionDate'] = df['InceptionDate'].apply(lambda x: x if x is None else pd.to_datetime(x).strftime('%m/%d/%y'))

        # region formatting configuration
        metrics2include = [('P&L(bps)', 'ITD'), ('P&L(bps)', 'YTD'), ('ROMC(bps)', 'YTD'), ('P&L(bps)', 'MTD'),
                           ('ROMC(bps)', 'MTD'), ('P&L(bps)', '5D'), ('P&L(bps)', '1D'),
                           ('P&L($)', 'ITD'), ('P&L($)', 'YTD'), ('P&L($)', 'MTD'), ('P&L($)', '5D'), ('P&L($)', '1D'),
                           ('ANN. VOL', '30D'), ('CAPITAL($)', '1D')]
        metric2display_name = {'P&L(bps)': '', 'P&L($)': '', 'ANN. VOL': 'VOL',
                               'ROMC(bps)':'ROMC', 'CAPITAL($)':'CAPITAL'}
        metric2unit = {'P&L(bps)': 'bps', 'P&L($)': '$', 'ANN. VOL': '%',
                       'ROMC(bps)': 'bps', 'CAPITAL($)':'%'} # will transform capital from $ to % later
        # endregion

        fund2vol_size_table, fund2vol_breakdown_by_tg = {}, {}
        funds = ["ARB", "AED", "TACO", "WED", "CAM", "LG", "TAQ", "LEV", "MACO", "MALT"]
        overall_df = pd.DataFrame()
        for fund_code in funds:
            f_df = df[df['Fund'] == fund_code].copy()
            fund_name = get_fund_code2name()[fund_code]
            f_nav = get_NAV_df(fund_name)
            f_curr_nav = f_nav['NAV'].iloc[-1]

            metrics_df = pd.DataFrame([json2row(json) for json in f_df['Metrics in Bet JSON']])
            metrics_df.index = f_df.index

            for (metric, period) in metrics2include:
                unit = metric2unit[metric]
                disp_name = metric2display_name[metric]
                display_colname = disp_name + ' ' + period + '(' + unit + ')'
                f_df[display_colname] = metrics_df[metric + '|' + period]

            f_df['CAPITAL 1D(%)'] = [np.nan if status == 'CLOSED' else 1e2*(cap_usd/f_curr_nav)
                                     for (cap_usd,status) in zip(f_df['CAPITAL 1D(%)'],f_df['Status'])]

            del f_df['Metrics in NAV JSON']; del f_df['Metrics in NAV notes JSON']
            del f_df['Metrics in Bet JSON']; del f_df['Metrics in Bet notes JSON']
            del f_df['Analyst']

            sleeve2code = {'Merger Arbitrage': 'M&A', 'Equity Special Situations': 'ESS',
                           'Opportunistic' : 'OPP', 'Forwards':'FWD', 'Credit Opportunities':'CREDIT'}

            f_df['Sleeve'] = f_df['Sleeve'].apply(lambda x: sleeve2code[x] if x in sleeve2code else x)
            f_df = f_df[(~pd.isnull(f_df[' YTD($)']))]  # don't show null ytds
            f_df = f_df.sort_values(by=' YTD($)')
            f_df['Date'] = today.strftime('%Y-%m-%d')
            
            f_df.rename(columns={' ITD(bps)': 'ITD(bps)', ' YTD($)': 'YTD(bps)', ' MTD(bps)': 'MTD(bps)',
                                 ' 5D(bps)': '5D(bps)', ' 1D(bps)': '1D(bps)', ' ITD($)': 'ITD($)', ' YTD($)': 'YTD($)',
                                 ' MTD($)': 'MTD($)', ' 5D($)': '5D($)', ' 1D($)': '1D($)'}, inplace=True)

            overall_df = overall_df.append(f_df)
    
            # region VOLATILITY CHARTS
            vol_cohorts = ["0%-5%", "5%-10%", "10%-15%", "15%-20%", "20%-25%", "25%-30%", "30%-35%",
                           "35%-40%", "40%-45%", "45%-50%", "50%+"]
            vol_cohorts2cnt = {v: 0 for v in vol_cohorts}
            vol_cohorts2tg_vol_pairs = {v: [] for v in vol_cohorts}
            for (tg, vol) in zip(f_df['TradeGroup'], f_df['VOL 30D(%)']):
                if 0 < vol <= 5:
                    vol_cohorts2cnt["0%-5%"] += 1
                    vol_cohorts2tg_vol_pairs["0%-5%"].append((tg, vol))
                if 5 < vol <= 10:
                    vol_cohorts2cnt["5%-10%"] += 1
                    vol_cohorts2tg_vol_pairs["5%-10%"].append((tg, vol))
                if 10 < vol <= 15:
                    vol_cohorts2cnt["10%-15%"] += 1
                    vol_cohorts2tg_vol_pairs["10%-15%"].append((tg, vol))
                if 15 < vol <= 20:
                    vol_cohorts2cnt["15%-20%"] += 1
                    vol_cohorts2tg_vol_pairs["15%-20%"].append((tg, vol))
                if 20 < vol <= 25:
                    vol_cohorts2cnt["20%-25%"] += 1
                    vol_cohorts2tg_vol_pairs["20%-25%"].append((tg, vol))
                if 25 < vol <= 30:
                    vol_cohorts2cnt["25%-30%"] += 1
                    vol_cohorts2tg_vol_pairs["25%-30%"].append((tg, vol))
                if 30 < vol <= 35:
                    vol_cohorts2cnt["30%-35%"] += 1
                    vol_cohorts2tg_vol_pairs["30%-35%"].append((tg, vol))
                if 35 < vol <= 40:
                    vol_cohorts2cnt["35%-40%"] += 1
                    vol_cohorts2tg_vol_pairs["35%-40%"].append((tg, vol))
                if 40 < vol <= 45:
                    vol_cohorts2cnt["40%-45%"] += 1
                    vol_cohorts2tg_vol_pairs["40%-45%"].append((tg, vol))
                if 45 < vol <= 50:
                    vol_cohorts2cnt["45%-50%"] += 1
                    vol_cohorts2tg_vol_pairs["45%-50%"].append((tg, vol))
                if vol > 50:
                    vol_cohorts2cnt["50%+"] += 1
                    vol_cohorts2tg_vol_pairs["50%+"].append((tg, vol))

            vol_cohort_size_pairs = {k: vol_cohorts2cnt[k] for k in vol_cohorts}
            for k in vol_cohorts2tg_vol_pairs:
                vol_cohorts2tg_vol_pairs[k].sort(key=lambda x: x[1])  # sort by vol

            #endregion
            fund2vol_size_table[fund_code] = vol_cohort_size_pairs
            fund2vol_breakdown_by_tg[fund_code] = vol_cohorts2tg_vol_pairs
        
        cols = list(overall_df.columns.values)
        cols.pop(cols.index('Date')) #Want to reorganize so the Date timestamp comes first
        overall_df = overall_df[['Date'] + cols]

    except Exception as e:
        print e

    return {'TradeGroup Performance Over Own Capital': overall_df,
            'Vol Distribution by Fund': fund2vol_size_table,
            'Vol Distribution by Fund Breakdown by TradeGroup': fund2vol_breakdown_by_tg
           }

In [38]:
res = get_tradegroup_performance_over_own_capital()

In [39]:
res["TradeGroup Performance Over Own Capital"]

,Date,Fund,Sleeve,TradeGroup,LongShort,InceptionDate,EndDate,Status,ITD(bps),YTD(bps),...,ROMC MTD(bps),5D(bps),1D(bps),ITD($),YTD($),MTD($),5D($),1D($),VOL 30D(%),CAPITAL 1D(%)
1094,2019-03-21 09:35:46.270,ARB,M&A,GG - NEM,Long,01/14/19,03/20/19,ACTIVE,-25.781576,-25.781576,...,377.734751,17.432134,-3.424365,-1.031702e+06,-1.031702e+06,1.005004e+06,33321.036700,-12645.900000,22.025711,2.088743
1134,2019-03-21 09:35:46.270,ARB,M&A,IDTI - RENESAS,Long,09/11/18,03/20/19,ACTIVE,310.357991,-69.006669,...,-37.704357,-39.437745,35.424047,5.245788e+04,-4.916999e+05,-1.506053e+05,-166744.634000,152811.470000,3.904871,2.439906
1263,2019-03-21 09:35:46.270,ARB,M&A,NFX - ECA REVERSAL,Short,11/07/18,02/25/19,CLOSED,-898.077378,-493.387517,...,NaN,NaN,NaN,-5.701438e+05,-3.968777e+05,NaN,NaN,NaN,4.588779,NaN
1415,2019-03-21 09:35:46.270,ARB,M&A,STBZ - CADE,Long,05/14/18,01/10/19,CLOSED,981.445808,-121.649161,...,NaN,NaN,NaN,1.602466e+06,-2.388415e+05,NaN,NaN,NaN,NaN,NaN
1466,2019-03-21 09:35:46.270,ARB,M&A,TWR NZ - SUN AU,Long,06/28/17,03/20/19,ACTIVE,-2638.696291,-548.260539,...,81.015099,515.719584,-276.038965,-1.251225e+06,-2.384832e+05,3.398071e+04,216129.179533,-123305.622725,38.113657,0.252655
1219,2019-03-21 09:35:46.270,ARB,M&A,MEG CN - HSE CN,Long,10/01/18,01/22/19,CLOSED,-4514.810949,-3915.208430,...,NaN,NaN,NaN,-3.020835e+05,-2.288247e+05,NaN,NaN,NaN,NaN,NaN
1295,2019-03-21 09:35:46.270,ARB,M&A,PACB - ILMN,Long,11/02/18,03/20/19,ACTIVE,-241.434507,-56.076423,...,65.991801,81.077084,-0.055539,-3.447806e+05,-2.005054e+05,7.027471e+04,84294.835300,-70.870000,13.121867,0.721743
970,2019-03-21 09:35:46.270,ARB,M&A,CELG - BMY,Long,01/03/19,03/15/19,CLOSED,-119.727753,-119.727753,...,245.912098,-26.603930,NaN,-1.959107e+05,-1.959107e+05,1.745497e+05,-19002.579800,NaN,20.439116,NaN
894,2019-03-21 09:35:46.270,ARB,M&A,ATHN - VERITAS REVERSAL,Short,11/30/18,02/01/19,CLOSED,-160.776240,-217.616873,...,NaN,NaN,NaN,-1.112913e+05,-1.929233e+05,NaN,NaN,NaN,NaN,NaN
1230,2019-03-21 09:35:46.270,ARB,M&A,MLNX - NVDA,Long,03/11/19,03/20/19,ACTIVE,-52.179640,-52.179640,...,-49.657154,-9.223788,18.010430,-1.457413e+05,-1.457413e+05,-1.457413e+05,-31392.193700,58515.619600,4.264872,1.837651


In [40]:
res["Vol Distribution by Fund"]

{'AED': {'0%-5%': 36,
  '10%-15%': 10,
  '15%-20%': 7,
  '20%-25%': 2,
  '25%-30%': 4,
  '30%-35%': 0,
  '35%-40%': 3,
  '40%-45%': 1,
  '45%-50%': 0,
  '5%-10%': 19,
  '50%+': 11},
 'ARB': {'0%-5%': 43,
  '10%-15%': 4,
  '15%-20%': 2,
  '20%-25%': 3,
  '25%-30%': 0,
  '30%-35%': 1,
  '35%-40%': 2,
  '40%-45%': 0,
  '45%-50%': 0,
  '5%-10%': 17,
  '50%+': 2},
 'CAM': {'0%-5%': 36,
  '10%-15%': 10,
  '15%-20%': 7,
  '20%-25%': 2,
  '25%-30%': 4,
  '30%-35%': 0,
  '35%-40%': 3,
  '40%-45%': 1,
  '45%-50%': 0,
  '5%-10%': 19,
  '50%+': 11},
 'LEV': {'0%-5%': 30,
  '10%-15%': 3,
  '15%-20%': 1,
  '20%-25%': 0,
  '25%-30%': 2,
  '30%-35%': 1,
  '35%-40%': 1,
  '40%-45%': 0,
  '45%-50%': 0,
  '5%-10%': 10,
  '50%+': 1},
 'LG': {'0%-5%': 36,
  '10%-15%': 10,
  '15%-20%': 7,
  '20%-25%': 2,
  '25%-30%': 4,
  '30%-35%': 0,
  '35%-40%': 3,
  '40%-45%': 1,
  '45%-50%': 0,
  '5%-10%': 19,
  '50%+': 11},
 'MACO': {'0%-5%': 40,
  '10%-15%': 4,
  '15%-20%': 2,
  '20%-25%': 3,
  '25%-30%': 0,
  '30%-3

In [41]:
res['Vol Distribution by Fund Breakdown by TradeGroup']

{'AED': {'0%-5%': [('ATHN - VERITAS', 0.1520081754),
   ('DNB - THOMAS LEE', 0.3845957427),
   ('HF - JLL', 0.39839058720000003),
   ('DJO GLOBAL MERGER ARB BONDS', 0.9414648066),
   ('REN 2020 M&A BONDS', 1.2113897468000001),
   ('BEL - MC FP', 1.7417736959),
   ('MB - VISTA', 1.8201155790999999),
   ('LLL - HRS', 1.8248434924999999),
   ('SUM CN - MS', 1.9389387693),
   ('AHSL SS - CVC', 2.0319275603),
   ('FDC 5.00 01/15/24 MERGER BONDS', 2.0776896974),
   ('ESL - TDG', 2.1051370088),
   ('JACFIN SPEC M&A', 2.196964797),
   ('MBFI - FITB', 2.3498363966999998),
   ('FDC 5.75 01/15/24 MERGER BONDS', 2.4037350599),
   ('USG - KNAUF', 2.4224809007),
   ('ARNC CONVERTS', 2.5563723478),
   ('LXFT - DXC', 2.6423879634),
   ('MANX LN - BASALT', 2.9226762353),
   ('TVPT - SIRIS CAPITAL', 3.0335045501),
   ('MOMENT LITIGATION BONDS', 3.041443353),
   ('AABA R/R', 3.1210963317),
   ('NLSN SPEC M&A', 3.3908410951),
   ('TRCO - NXST', 3.5423608722),
   ('WP - FIS', 3.6110240208),
   ('ONCE - ROG